In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, mne, json, h5py, pymatreader, re, time, gc;
import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mpr
import matplotlib as mpl
import scipy.signal as sig
import pandas as pd 
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

from dateutil import parser
from utils_postprocess_HPC import collectCalcResults

from utils_postprocess_HPC import getMocFromRow, getMocFromRowMultiOPR

from pprint import pprint as ppr

In [ ]:
# subdir_res = 'collected'
# collected_dir = pjoin(gv.data_dir_tmp, subdir_res)

# read_saved = True
# save = False

# ####################
# fnf = pjoin(collected_dir, 'df_collected_medcondsep_rest&entire_prescale,scale=medcond.pkl')
# print(fnf)
# if read_saved:
#     df = pd.read_pickle(fnf)
# elif save:
#     df.to_pickle(fnf)
# ################

In [ ]:
subdir = 'for_Alfons' # for output

subdir_res = 'collected'
temp_dir = pjoin( os.path.expandvars('$SCRATCH') , 'OSCBAGDIS', subdir_res )
#pjoin(gv.data_dir_tmp, 'collected')

read_saved = True
#save       = False

#pptypes_to_load = ['covmat_entire', 'covmat_rest']
#pptypes_to_load = [ 'covmat_rest']
pptypes_to_load = [ 'SSS_covmat_rest']
#pptypes_to_load = [ 'covmat_rest', 'SSS_covmat_rest']

####################
import globvars as gv
if read_saved:
    pptype2res = {}
    #df = pd.read_pickle(fnf)
    
    for ppt in pptypes_to_load:
        fn = f'{ppt}_output_per_raw.npz'
        fnf = pjoin(temp_dir, fn)
        #print(fnf)
        f = np.load(fnf, allow_pickle=1)
        pptype2res[ppt] = f['arr_0'][()]
        del f
    import gc; gc.collect()
# elif save:
#     df.to_pickle(fnf)
################

In [ ]:
# we use loaded data so it should be fast
use_tmpdir_to_load = True
perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
to_show += [('trem_vs_2class','merge_movements','basic')]
to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]
from utils_postprocess import  multiLevelDict2TupleList

dfs = []
for pptype,opr in pptype2res.items():
    outputs_grouped_tpll = multiLevelDict2TupleList(opr,4,3)
    outputs_grouped_tpll[0][:-1]
    dfunproc = pd.DataFrame(outputs_grouped_tpll, 
        columns = ['rawname','prefix','grouping','interval_set','mcf'] )
    prefixes = list( dfunproc['prefix'].unique() )

    # warnings.simplefilter('error')
    dftmp, table_info_per_perf_type, table_per_perf_type = \
        postp.prepTableInfo3(opr, prefixes=prefixes, 
        perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True,
        use_tmpdir_to_load = use_tmpdir_to_load)
    gc.collect()
    dftmp['pptype'] = pptype
    
    dfs += [dftmp]

df_noext = pd.concat(dfs, ignore_index=True)
import gc; gc.collect()

In [ ]:
#from utils_postprocess_HPC import extendDF
from utils_postprocess_HPC import extendDfMultiOPR
#%debug
df2 = extendDfMultiOPR(df_noext,pptype2res)
df = df2

In [ ]:
#preferred_pptype = 'covmat_rest'
preferred_pptype = 'SSS_covmat_rest'
output_per_raw = pptype2res[preferred_pptype]
df_all = df.query(f'pptype == "{preferred_pptype}"').copy()

prefix_templs = df_all['prefix_templ'].unique()
prefix_templs

In [ ]:
import gc; gc.collect()

In [ ]:
# I think it is obsolete -- I can get this info directly from moc
from utils import formatMultiRawnameStr
from utils_postprocess import getBestLFPfromDict

row = df.to_dict('records')[2]
moc = getMocFromRow( row , output_per_raw)

best_LFP_info_fname_full = pjoin(gv.data_dir,  moc['pars']['best_LFP_info_file'])
with open(best_LFP_info_fname_full, 'r') as f:
    best_LFP_info = json.load(f)

def extractBestLFPaddInfo(row):
    # this repeats what happens inside run_ML in this respect
    moc = getMocFromRow( row , output_per_raw)
    if moc is None:
        print(row['prefix'],row['grouping'],row['interval_set'],'no moc')
        return None,None,None,None,None
    pars = moc['pars']
    best_LFP_data_mods = ['LFP']
    best_LFP_featstr = 'onlyH_act'
    best_LFP_exCB = None
    subj = row['subject']
    data_modalities = pars['mods'].split(',')
    disjoint = -int( pars['subskip_fit'] )
    savefile_rawname_format = pars['savefile_rawname_format']
    custom_rawname_str = pars.get('custom_rawname_str',None)
    groupings_to_use = [row['grouping']]
    int_types_to_use = [row['interval_set']]

    rawnames = pars['rawnames'].split(',')
    brain_side_to_use = pars['brain_side_to_use']

    if tuple(best_LFP_data_mods) == tuple(['LFP']):
            bestLFP_prefix_type = f'modLFP_{best_LFP_featstr}'
    elif set(best_LFP_data_mods) == set(['LFP','msrc']):
        bestLFP_prefix_type = best_LFP_featstr
    else:
        raise ValueError('wrong bestLFP_prefix_type = {bestLFP_prefix_type}')
    if best_LFP_exCB is None:
        if brain_side_to_use.startswith('both'):
            exCB = False
        elif tuple(data_modalities) == tuple(['LFP']) or \
            tuple(best_LFP_data_mods) == tuple(['LFP']):
            exCB = False
        else:
            exCB = True
    else:
         exCB = best_LFP_exCB

    rncombinstr = formatMultiRawnameStr(rawnames, savefile_rawname_format,
             custom_rawname_str = custom_rawname_str)

     # TODO: load this file and check that labels are the same
    mainLFPchan, fnf_searchLFPres, g =\
             getBestLFPfromDict(best_LFP_info,subj,rncombinstr,
             disjoint=disjoint,
             brain_side = brain_side_to_use,
             exCB = exCB,
             grp=groupings_to_use[0],
             it=int_types_to_use[0])

    (g,prefix_type,side_det_str,grp,it,best_kind) = g
    return prefix_type,side_det_str,grp,it,best_kind
df_all[['bestLFP_prefix_type','bestLFP_side_det_str',
        'bestLFP_grouping','bestLFP_interval_set','bestLFP_best_kind']] = \
    df_all.apply(extractBestLFPaddInfo,1,result_type='expand')

In [ ]:
preferred_pptype, subdir

In [ ]:
best_LFP_cols = ['bestLFP_prefix_type','bestLFP_side_det_str',
        'bestLFP_grouping','bestLFP_interval_set','bestLFP_best_kind']

from utils_postprocess_HPC import addImprovColDfAll, findBestParcelGroups
base_pref_start_dict = {'onlyH_subskip': 'onlyH_modLFP_subskip',
 'onlyH_act_subskip': 'onlyH_act_modLFP_subskip',
 'onlyH_act_LFPand_only': 'onlyH_act_modLFP_subskip8',
 'onlyH_act_only': 'onlyH_act_modLFP_subskip8'}

df_all_allsep = df_all[(df_all['grouping'] == 'merge_nothing') \
                       & (df_all['interval_set'] == 'basic')].copy()

addImprovColDfAll(df_all, inplace = True, 
                      score_shuffled = 'bacc_perm_0.9',
                      base_pref_start_dict=base_pref_start_dict )

In [ ]:
df_all_allsep = df_all[(df_all['grouping'] == 'merge_nothing') \
                       & (df_all['interval_set'] == 'basic')].copy()
dfstat = findBestParcelGroups(df_all_allsep)
import gc; gc.collect()

gs = ','.join( df_all['grouping'].unique().astype(str) )

#fn ='df_collected_medcondsep_rest&entire_prescale,scale=medcond.pkl'
# fn ='df_collected_medcondsep_rest&entire_prescale,scale=medcond.pkl'
# fnf = pjoin(gv.data_dir,subdir_res,fn)

dfstat.to_pickle( pjoin(gv.fig_dir,subdir,f'{preferred_pptype}_dfstat_parcel_groups_{gs}.pkl') )
df_all.to_pickle( pjoin(gv.fig_dir,subdir,f'{preferred_pptype}_df_all_{gs}.pkl') )

In [ ]:
TODO:
for each beh state confmat diagonal elements
mislabeling

# Table 2

In [ ]:
fnf = moc['filename_full']
r = np.load(fnf, allow_pickle=1)

In [ ]:
ts = r['rawtimes_pri'][()]
ts[0][-1] + ts[1][-1]

In [ ]:
#print(list(r.keys()))
#['results_cur', 'label_encoder', 'Xconcat_good_cur', 'class_labels_good', 'selected_feat_inds_pri', 
# 'feature_names_filtered_pri', 'featnames_nice', 'bininds_good', 'inds_not_neut', 'feat_info_pri', 
# 'rawtimes_pri', 'Xtimes_pri', 'wbd_pri', 'pcapts', 'pcaobj', 'icaobj', 'X_imputed', 
# 'pars', 'roi_labels', 'ann_related']

In [ ]:
moc['counts']['trem_L'] / 8

In [ ]:
moc['counts'], moc['pars']['rawnames']

In [ ]:
cols_pctpts = []
for it in gp.int_types_basic:
    cols_pctpts += [f'numpts_{it}']
cols_pctpts

In [ ]:

prefix_cur_table = 'onlyH_act'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"').copy()

# it is a bit long so I don't want to put in in extendDf ran for all rows
def lbd(row):
    moc = getMocFromRow(row, output_per_raw)
    fnf = moc['filename_full']
    r = np.load(fnf, allow_pickle=1)
    ts = r['rawtimes_pri'][()]
    tt = ts[0][-1] + ts[1][-1]
    return tt
df_all_sub['duration'] = df_all_sub.apply(lbd,1)

#'rawname'
srt = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])
r = srt[['prefix_templ', 'subject', 'medcond','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ', 'duration']]

display(r)
rr = r.groupby(['prefix_templ','medcond'])
#rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

In [ ]:
# reshape slignely and export to excel
res = srt[['medcond','subject'] + cols_pctpts + ['duration','bacc']].copy()
    
me = rr.mean(numeric_only=1).reset_index()
me['subject'] = 'mean'
me = me.drop(['numpts','prefix_templ'],1)

std = rr.std(numeric_only=1).reset_index()
std['subject'] = 'std'
std = std.drop(['numpts','prefix_templ'],1)

res = pd.concat( [res, me, std], ignore_index=1)

for col in cols_pctpts:
    res[col ] = (res[col ] )  .astype('float')

    
res = res.rename(columns={'numpts_trem':'tremor', 'numpts_notrem':'rest',
                     'numpts_hold':'hold', 'numpts_move':'grasp'})
res['bacc'] = res['bacc'].apply(lambda x: f'{x * 100:.0f}%')
res = res.sort_values(['medcond','subject'])
display(res)


res.to_excel(pjoin(gv.fig_dir,subdir,'table2.xlsx'))

## get mean number of points and dur

In [ ]:
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ == {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')

if 'duration' not in df_all_sub.columns:
    def lbd(row):
        moc = getMocFromRow(row, output_per_raw)
        fnf = moc['filename_full']
        r = np.load(fnf, allow_pickle=1)
        ts = r['rawtimes_pri'][()]
        tt = ts[0][-1] + ts[1][-1]
        return tt
    df_all_sub['duration'] = df_all_sub.apply(lbd,1)

r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','numpts','numpts2','duration']]
rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

### t

In [ ]:
best_LFP_cols2 = ['bestLFP_prefix_type',
 'bestLFP_side_det_str',
 'bestLFP_best_kind']

In [ ]:
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['subject', 'medcond', 'prefix_templ', 'bacc',
      'mainLFPside', 'mainLFPside_templ'] + best_LFP_cols2  ]
display(r)

In [ ]:
cols_pctpts =  ['pctpts_trem','pctpts_notrem','pctpts_move','pctpts_hold', 'numpts']

In [ ]:
prefix_templs

In [ ]:
df_all.columns

In [ ]:
cols_pctpts = []
prefix_cur_table = 'onlyH_act'
prefl = [f'{prefix_cur_table}_subskip8@B-B', f'{prefix_cur_table}_subskip8@%_exCB-%', 
                f'{prefix_cur_table}_subskip8@^_exCB-^']
df_all_sub = df_all.query(f'prefix_templ == {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
display(r)
rr = r.groupby(['prefix_templ','medcond'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

In [ ]:
rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

In [ ]:
f'{prefix_cur_table}_subskip8BC'

In [ ]:
moc = getMocFromRow( dict( df_all.loc[6351] ), output_per_raw)
moc['filename_full']

In [ ]:
moc['cmd']

In [ ]:
# TODO: cahnge rungenstrings 
# TODO

In [ ]:
per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest/_S04_off_parcel_aal_grp10-0_onlyH_act_subskip8BC_ML_nr2_7chs_nfeats*skip32_wsz256_mainLFP__(merge_nothing,basic).npz

In [ ]:
df_all['prefix'].str.contains?

In [ ]:
#df_all.loc[ df_all['prefix'].str.contains('only') , 'prefix' ]

### solve wrong CB prefix handling problem

In [ ]:
c = (df_all['num'] < 30) & (~df_all['prefix'].str.contains('_only') ) &\
                            (~df_all['prefix'].str.contains('modLFP') )

prefs_vulnerable = df_all.loc[c,'prefix'].unique()
print( prefs_vulnerable )

In [ ]:
df_all.loc[c,['subject', 'medcond','bacc','prefix', 'mainLFPside',
    'mainLFPside_templ','bestLFP_side_det_str','num']]

In [ ]:
import glob
newrows = []
ctr = 0
for rowi,row in df_all.loc[c].iterrows():
    
    moc = getMocFromRow(row, output_per_raw)
    fnf = moc['filename_full']
    r = re.match( '.*(_nfeats[0-9]+_pcadim[0-9]+_).*' , fnf )
    sp = r.groups()[0]
    matchstr = fnf.replace(sp, '_nfeats*_pcadim*_')
    
    fnfs = glob.glob(matchstr)
    assert len(fnfs) == 2
    print(ctr, fnfs)
    
    #rcs = []
    for fnf_cur in fnfs:
        f = np.load(fnf_cur, allow_pickle=True)
        rc = f['results_cur'][()]
        #rcs += []
        del f 

        #print( rc['cmd'] )
        pars = rc['pars']
        print(pars['brain_side_to_use'])
        pref = pars['prefix']
        if pars['brain_side_to_use'] == 'both':
            newpref = pref
        elif pars['brain_side_to_use'] == 'both_onlyCB':
            newpref = pref.replace('subskip8BC', 'subskip8bothOnlyCB_C')
        else:
            raise ValueError(f"unexpected {pars['brain_side_to_use']}")
            
        rc['pars']['prefix'] = newpref

        if newpref not in output_per_raw[row['rawname']]:
            output_per_raw[row['rawname']][newpref] = {}
        if row['grouping'] not in output_per_raw[row['rawname']][newpref]:
            output_per_raw[row['rawname']][newpref][row['grouping']] = {}
            
        rc['filename_full'] = fnf_cur
        rc['loaded'] = True
        output_per_raw[row['rawname']][newpref][row['grouping']][row['interval_set']] = rc

        import gc; gc.collect()
    ctr += 1
    import gc; gc.collect()
    
    

In [ ]:
for rowi,row in df_all.loc[c].iterrows():
    output_per_raw[row['rawname']][row['prefix']][row['grouping']][row['interval_set']]['loaded'] = True

In [ ]:
tpll = multiLevelDict2TupleList(output_per_raw,4,3)
dftmp = pd.DataFrame(tpll, columns = ['rawname','prefix','grouping','interval_set','mcf'] )
c = dftmp[ 'prefix'].isin( prefs_vulnerable) |  dftmp['prefix'].str.contains('bothOnlyCB')
prefs_to_recollect = dftmp.loc[c,'prefix'].unique()

In [ ]:
prefs_to_recollect

In [ ]:
#dfext

In [ ]:
df1 = pd.concat([df,dfext])

In [ ]:
from utils_postprocess_HPC import extendDf
dfext = extendDf(dfext, output_per_raw)

In [ ]:
perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
to_show += [('trem_vs_2class','merge_movements','basic')]
to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]

# warnings.simplefilter('error')
dfext, table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefs_to_recollect, 
    perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True)
gc.collect()


In [ ]:
list(set())

In [ ]:
df_all_recol, table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=df_all['prefix'].unique().tolist(), 
    perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True)
gc.collect()

In [ ]:
output_per_raw['S07_on']['onlyH_mob_subskip8RC']['merge_nothing']['basic'].keys()

In [ ]:
moc['filename_full'].replace()

In [ ]:
moc['featnames_for_fit']

In [ ]:
prefix_cur_table = 'onlyH_act'
df_all_sub = df_all.query(f'prefix == "{prefix_cur_table}_subskip8BC" and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject'])[['subject', 'medcond','bacc',
                                                      'points_per_beh_state', 'mainLFPside',
                                                      'mainLFPside_templ','bestLFP_side_det_str','num']]
display(r)
rr = r.groupby(['medcond'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

# Tables and stats for text input

## for abstract (across medcond)

##### both sides H_act

In [ ]:
cols_pctpts = []
prefix_cur_table = 'onlyH_act'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
display(r)
#rr = r.groupby(['prefix_templ','medcond'])
rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

##### modLFP H_act

In [ ]:
cols_pctpts = []
prefix_cur_table = 'onlyH_act_modLFP'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
display(r)
#rr = r.groupby(['prefix_templ','medcond'])
rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

## for Results

### STN alone

In [ ]:
# T-test
cols_pctpts = []
prefix_cur_table = 'onlyH_act_modLFP'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
display(r)
#rr = r.groupby(['prefix_templ','medcond'])
#rr = r.groupby(['prefix_templ'])
#print('mean = ') 
#display( rr.mean(numeric_only=1) )
#print('std = ')
#display( rr.std(numeric_only=1) )
####################
from scipy.stats import ttest_ind

mc2v = {}
for medcond in ['off','on']:
    mc2v[medcond ] = r.query('medcond == @medcond') ['bacc'].values
print( mc2v )
print('Diff beteween off and on')
ttest_ind(mc2v['off' ], mc2v['on' ])

#### Merging beh states STN-alone

In [ ]:
df_all['grouping'].unique()

In [ ]:
# merge merge_all_not_trem
cols_pctpts = []
prefix_cur_table = 'onlyH_act_modLFP'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_all_not_trem" and interval_set == "basic"')
#'rawname'
# grouping just so that we see it
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'grouping', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
#display(r)
rr = r.groupby(['prefix_templ','medcond','grouping'])
#rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

In [ ]:
# merge trem_vs_quiet
cols_pctpts = []
prefix_cur_table = 'onlyH_act_modLFP'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "trem_vs_quiet"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','interval_set','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
#display(r)
rr = r.groupby(['prefix_templ','medcond','interval_set'])
#rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

### STN + cortex

#### Average performance gain

In [ ]:
#Average performance gain
cols_pctpts = []
prefix_cur_table_LFP = 'onlyH_act_modLFP'
prefl = [f'{prefix_cur_table_LFP}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')

r_LFP = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [[ 'subject', 'medcond','bacc']]
#display(r_LFP)

prefix_cur_table = 'onlyH_act'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')

r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [[ 'subject', 'medcond','bacc']]

cols = ['subject', 'medcond']
r = r.rename(columns={'bacc':'bacc_LFP&CTX'})
rm = pd.merge(r.set_index(cols), r_LFP.set_index(cols), on=cols)
display(rm)
rm = rm.reset_index()
#display(rm)
rm['impr'] = rm['bacc_LFP&CTX'] - rm['bacc']
print('Average performance gain')
rm['impr'].mean()

#### T-test

In [ ]:
# T-test
cols_pctpts = []
prefix_cur_table = 'onlyH_act'
prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping == "merge_nothing" and interval_set == "basic"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
    [['prefix_templ', 'subject', 'medcond','pptype','bacc'] + cols_pctpts + \
      ['numpts', 'mainLFPside', 'mainLFPside_templ']]
display(r)
#rr = r.groupby(['prefix_templ','medcond'])
#rr = r.groupby(['prefix_templ'])
#print('mean = ') 
#display( rr.mean(numeric_only=1) )
#print('std = ')
#display( rr.std(numeric_only=1) )
####################
from scipy.stats import ttest_ind

mc2v = {}
for medcond in ['off','on']:
    mc2v[medcond ] = r.query('medcond == @medcond') ['bacc'].values
print( mc2v )
print('Diff beteween off and on')
ttest_ind(mc2v['off' ], mc2v['on' ])

### Multi table

In [ ]:
df_all['bacc_perm_0.9']

In [ ]:
df_all['interval_set'].unique()

In [ ]:
dd[('merge_nothing','basic')]

In [ ]:
prefix_cur_table_LFP = 'onlyH_act_modLFP'
prefix_cur_table = 'onlyH_act'
grps = ['merge_nothing', 'merge_all_not_trem']
prefl = [f'{prefix_cur_table_LFP}_subskip8@B-B', f'{prefix_cur_table}_subskip8@B-B' ]
dfc = df_all.query(f'prefix_templ in {prefl} and '
                          'grouping in @grps').copy()


dd = {}
dd[('merge_nothing','basic')] = 'all vs. all'
dd[('merge_all_not_trem','basic')] = 'trem vs. all other'
dd[('merge_nothing','trem_vs_quiet')] = 'trem vs. rest'
def lbd(row):
    tpl = row['grouping'], row['interval_set']
    return dd[tpl]
dfc['Classification'] = dfc.apply(lbd,1)

cols = ['prefix_templ','medcond','Classification']
dfc = dfc[cols + ['bacc', 'bacc_perm_0.9']]

grp = dfc.groupby(['prefix_templ','medcond','Classification'])

std = grp.std(numeric_only=1).reset_index()
#std = std.drop(['bacc_perm_0.9'],1)
std = std.rename(columns = {'bacc':'std', 'bacc_perm_0.9':'perm_std'})

me = grp.mean(numeric_only=1).reset_index()
me = me.rename(columns = {'bacc':'mean', 'bacc_perm_0.9':'perm_mean'})
# r_LFP = df_all_sub.sort_values(by=['medcond','subject','prefix_templ'])\
#     [[ 'subject', 'medcond','bacc','bacc_perm_0.9']]

dfc2 = pd.merge( me.set_index(cols), std.set_index(cols), on=cols ).reset_index()
dfc2

dfc2_m =dfc2.copy()
dfc2_m = dfc2_m.drop(labels=['perm_mean','perm_std'],axis=1)
dfc2_m['shuffling'] = 'no'
dfc2_m

dfc2_s = dfc2.copy()
dfc2_s = dfc2_s.drop(['mean', 'std'],1)

dfc2_s = dfc2_s.rename(columns = {'perm_mean':'mean', 'perm_std':'std'})
dfc2_s['shuffling'] = 'yes'

dfc3 = pd.concat([dfc2_m, dfc2_s], ignore_index=1 ).reset_index()

In [ ]:
r.to_excel(pjoin(gv.fig_dir,subdir,'multitable_perfs.xlsx'))

In [ ]:
r = dfc3.sort_values(['prefix_templ','medcond','Classification'] )\
    [['prefix_templ','Classification','medcond','shuffling','mean','std']]
r

In [ ]:
# concat horizontally
# compute improv per dataset

### supp (one-sided one feat and all Hjorth)

#### contralat H_act

In [ ]:

prefix_cur_table = 'onlyH_act'
df_all_sub = df_all.query(f'prefix_templ == "{prefix_cur_table}_subskip8%%" and '
                          'grouping == "merge_nothing" and interval_set == "basic"'
                         ' and pptype == "covmat_rest"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject'])[['subject','medcond','pptype','bacc',
                                                      'points_per_beh_state', 'num']]
display(r)
rr = r.groupby(['medcond'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

#### contralat all Hjorth

In [ ]:
prefix_cur_table = 'onlyH'
df_all_sub = df_all.query(f'prefix_templ == "{prefix_cur_table}_subskip8%%" and '
                          'grouping == "merge_nothing" and interval_set == "basic"'
                         ' and pptype == "covmat_rest"')
#'rawname'
r = df_all_sub.sort_values(by=['medcond','subject'])[['medcond','pptype','bacc']]
display(r)
rr = r.groupby(['medcond'])
rr.mean(numeric_only=1),rr.std(numeric_only=1)

### Compare main stats with other feature sets

#### Create sub df

In [ ]:
use_prefix = False
#dfsub = df[['rawname','prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled']]
dfsub = df[['rawname','grouping','interval_set','pptype',
            'prefix','prefix_templ', 'num','numpts','bacc','bacc_perm_0.9','sens','spec',
           'tremor_det_perf']]

prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8','onlyH_mob_subskip8','onlyH_compl_subskip8','allb_beta_noH_subskip8','allb_gamma_noH_subskip8','allb_tremor_noH_subskip8',
        'LFPrel_noself_subskip8','modSrc_self_subskip8','modSrc_subskip8','modLFP_subskip8']

if use_prefix:
    prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8',
                        'onlyH_mob_subskip8','onlyH_compl_subskip8']
    prefix_role = 'prefix'
else:
    pref_templs_to_compare = ['onlyH_subskip8%%','onlyH_act_subskip8%%',
                        'onlyH_mob_subskip8%%','onlyH_compl_subskip8%%',
                             'onlyH_modLFP_subskip8%%']
    prefix_role = 'prefix_templ'

dfsub2 = dfsub[ dfsub[prefix_role].isin(pref_templs_to_compare) ]

#dfsub2 = dfsub2[ (dfsub2['grouping'] == grp) & (dfsub2['interval_set'] == it) ]
grp =  dfsub2.groupby( by=['pptype', prefix_role,'grouping','interval_set'] )

cts = set( list( grp.count()['rawname'] ) ) 
assert tuple( cts)  == tuple([11]), cts


#print ( len( dfsub2.groupby('prefix') ) )
dfmn = grp.mean().reset_index()
dfstd = grp.std().reset_index()

rename_dict = dict( zip( list( dfstd.columns), [col + '_std' for col in  dfstd.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
del rename_dict['pptype']
dfstd = dfstd.rename(columns=rename_dict)

rename_dict = dict( zip( list( dfmn.columns), [col + '_mean' for col in  dfmn.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
del rename_dict['pptype']
dfmn = dfmn.rename(columns=rename_dict)

dfstat = dfmn
for col in dfstd.columns:
    if col in ['prefix_templ','prefix','num_std','grouping','interval_set']:
        continue
    dfstat[col] = dfstd[col]
cols_desired = ['pptype',prefix_role, 'grouping','interval_set','bacc_mean','bacc_std',
                      'tremor_det_perf_mean','tremor_det_perf_std', 'num_mean']
dfstat_short = dfstat[cols_desired]

display( dfstat_short )

In [ ]:
import gc; gc.collect()

#### Hjorth activity optimality (within other Hjorth) -- Table 3

In [ ]:
# optimality
dfopt = dfmn[(dfmn['grouping']=='merge_nothing') & (dfmn['interval_set']=='basic')].copy()

cols_desired = [prefix_role, 'tremor_det_perf_mean','tremor_det_perf_std',
               'bacc_mean','bacc_std','num_mean']
dfopt = dfopt[cols_desired]

dfopt['bacc_over_num'] = dfopt.apply(lambda row: \
                        f'{row.bacc_mean / row.num_mean:.3f}', axis=1)
dfopt.sort_values('bacc_over_num')

pctf = lambda x: f'{x * 100:.0f}%'
dfopt['num_mean'] = dfopt['num_mean'].map(int)
dfopt['bacc_mean'] = dfopt['bacc_mean'].apply(pctf )
dfopt['bacc_std'] = dfopt['bacc_std'].apply(pctf )
dfopt['tremor_det_perf_mean'] = dfopt['tremor_det_perf_mean'].apply(pctf )
dfopt['tremor_det_perf_std'] = dfopt['tremor_det_perf_std'].apply(pctf )

###############
for s in dfopt[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfopt[prefix_role].replace(rename_dict, inplace=True )

rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity'}
dfopt[prefix_role].replace(rename_dict, inplace=True )

rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc_mean'] = 'performance mean'
rename_dict['bacc_std'] = 'performance std'
rename_dict['tremor_det_perf_mean'] = 'tremor detection mean'
rename_dict['tremor_det_perf_std'] = 'tremor detection std'
rename_dict['num_mean'] = 'feature number'        
#
rename_dict['bacc_over_num'] = 'mean pef. / num. features'        
dfopt.rename(columns=rename_dict, inplace=True)



dfopt.style.set_properties(**{'text-align': 'left'})

#display( dfopt )

#### Compare groupings

In [ ]:
show_only_act = 1
if show_only_act:
    if prefix_role == 'prefix':
        good = 'onlyH_act_subskip8'
    else:
        good = 'onlyH_act_subskip8%%'
    dfstat_short_nice = dfstat_short[dfstat_short[prefix_role] == good].copy()
    dfstat_short_nice = dfstat_short_nice.drop(['num_mean'], axis=1)
else: # all Hjorth
    dfstat_short_nice = dfstat_short.copy()

rename_dict = {}
for s in dfstat_short_nice[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )


rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity'}
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )

rename_dict = { 'merge_all_not_trem':'all non-tremor states', 
            'merge_movements':'voluntary movements',
            'merge_nothing':'none'          }
dfstat_short_nice['grouping'].replace(rename_dict, inplace=True )

rename_dict = { 'basic':'tremor,quiet,hold,grasp', 
            'trem_vs_quiet':'tremor,quiet'  }
dfstat_short_nice['interval_set'].replace(rename_dict, inplace=True )

pctf = lambda x: f'{x * 100:.0f}%'
if not show_only_act:
    dfstat_short_nice['num_mean'] = dfstat_short_nice['num_mean'].map(int)
dfstat_short_nice['bacc_mean'] = dfstat_short_nice['bacc_mean'].apply(pctf )
dfstat_short_nice['bacc_std'] = dfstat_short_nice['bacc_std'].apply(pctf )
dfstat_short_nice['tremor_det_perf_mean'] = dfstat_short_nice['tremor_det_perf_mean'].apply(pctf )
dfstat_short_nice['tremor_det_perf_std'] = dfstat_short_nice['tremor_det_perf_std'].apply(pctf )

# dfstat_short_nice['performance / feature number'] = \
#     dfmn.apply(lambda row: f'{row.bacc_mean / row.num_mean:.4f}', axis=1 )

# rename column names
rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc_mean'] = 'performance mean'
rename_dict['bacc_std'] = 'performance std'
rename_dict['tremor_det_perf_mean'] = 'tremor detection mean'
rename_dict['tremor_det_perf_std'] = 'tremor detection std'
if not show_only_act:
    rename_dict['num_mean'] = 'feature number'
dfstat_short_nice.rename(columns=rename_dict, inplace=True)

columns = np.array(list(dfstat_short_nice.columns) )[[0,1,2,5,6,3,4]]
columns

dfstat_short_nice[columns]

In [ ]:
suff = '_subskip8'
d['onlyH'] =  'Hjorth'
for freq in ['tremor','beta','gamma']:
    d[f'allb_{freq}'] = f'{freq} interactions and band power w/o Hjorth'

In [ ]:
dfstat_short[dfstat_short['prefix'] == 'onlyH_act_subskip8']

In [ ]:
dfstat_short_nice.columns

In [ ]:
list(dfstat_short_nice.columns)

In [ ]:
We have compared 11 different feature sets: all Hjorth parameters together 

In [ ]:
dfstat_short

In [ ]:
# % -- contra
# ^ -- ipsi



In [ ]:
#### Save perftable
ss = []
for tpl in to_show:
    ss += [','.join(tpl)]
s = ':'.join(ss)
print(s)

fn_table = pjoin(gv.data_dir,subdir, f'perftable_{s}_{len(prefixes)}prefs.npz')
print(fn_table)
svd = dict(table_info_per_perf_type=table_info_per_perf_type, table_per_perf_type=table_per_perf_type)
np.savez(fn_table, **svd)

In [ ]:
perf_to_use_list

## best and worst LFP vs cortical, read best LFP json

In [ ]:
best_LFP_info_file = 'searchLFP_both_sides_oversample2/best_LFP_info_both_sides_ext.json'
best_LFP_info_fname_full = pjoin(gv.data_dir, best_LFP_info_file)
print(f'Best LFP fname = {best_LFP_info_fname_full}')
with open(best_LFP_info_fname_full, 'r') as f:
    best_LFP_info = json.load(f)


In [ ]:
pp.printDict(best_LFP_info['S01'])

In [ ]:
rows = []
ps = []
for subj,d0 in best_LFP_info.items():
    for ks,d1 in d0.items():
        row = {}
        (prefix,grouping,iset) = ks.split(',')
        ps += [prefix]
        d2  = d1['balanced_accuracy']
        d2s = d1['balanced_accuracy_shuffled']
        
        chn = d2['best_LFP']['only']
        chn_cl = d2['best_LFP_contralat_to_move']['only']
        
        p    = d2['all_present_features']
        pb   = d2[f'all_present_features_but_{chn}']
        pbcl = d2.get(f'all_present_features_but_{chn_cl}',None)    
        
        p_s    = d2s['all_present_features']
        pb_s   = d2s[f'all_present_features_but_{chn}']
        pbcl_s = d2s.get(f'all_present_features_but_{chn_cl}',None)
        #d0['balanced_accuracy']['all_present_features']
        row['prefix_long'] = prefix
        row['grouping'] = grouping
        row['interval_set'] = iset
        row['subject'] = subj
        row['bacc_all'] = p
        row['bacc_best'] = pb
        row['bacc_best_contra'] = pbcl
        
        row['runCID'] = d1['runCID']
        
        row['bacc_all_shuffled']         = p_s
        row['bacc_best_shuffled']        = pb_s
        row['bacc_best_contra_shuffled'] = pbcl_s
        
        
        row['num'] = d1['total_num_feats']
        row['numpts'] = d1['total_num_datapoints']
        row['fname_mod_time'] = d1['fname_mod_time']
        
        parts = prefix.split('_')
        row['modality'] = parts[0]
        
        if parts[-1].find('disj') >= 0:
            row['disj'] = parts[-1]             
            i = -2
        else:
            row['disj'] = 'no'
            i = -1
            
        row['brain_side'] = parts[i][len('brain'):] 
        row['feat'] = '_'.join( parts[1:i])
        
        
        row['best_chn'] = chn
        row['best_chn_contra'] = chn_cl
        
        sidelet = row['brain_side'][0].upper()
        if sidelet == 'B':
            sidetempl = 'BB'
        else:
            sidetempl = sidelet2sideTempl( sidelet, subj )
            sidetempl = sidetempl + sidetempl
        if row['disj'] == 'disjoint':
            subskip = 8
        elif row['disj'] == 'semidisjoint':
            subskip = 4
        row['prefix_templ'] = row['modality'] + \
            f'_{row["feat"]}_' + f'subskip{subskip}' + sidetempl
        
        rows += [row]
#         break
#     break
dfLFP = pd.DataFrame(rows)

In [ ]:
# select disjoint and restrucutre the table
dfLFPd = dfLFP[(dfLFP['disj'] == 'disjoint') & (dfLFP['grouping'] == 'merge_nothing') \
              & (dfLFP['interval_set'] == 'basic')]

grp = dfLFPd.groupby(['prefix_templ','grouping','interval_set'])
display(grp.size())
dfLFPdm = grp.mean()
dfLFPdstd = grp.std()
display(dfLFPdm,dfLFPdstd)

dfLFPdm = dfLFPdm.reset_index()
dfLFPdstd = dfLFPdstd.reset_index()

finals = []

prefsd = {'prefix':{}, 'ending':{}, 'prefix_short':{}}
prefsd['ending']['contra'] = '_subskip8%%'
prefsd['ending']['ipsi'] = '_subskip8^^'
for s,end in prefsd['ending'].items():
    prefsd['prefix'][s] = [f'modLFP_onlyH{end}',f'modLFP_onlyH_act{end}']
    prefsd['prefix_short'][s] = [f'onlyH_act{end}', f'onlyH{end}']    

    # now I want to make two tables each taking two rows
    dfLFPdm_sub = dfLFPdm[dfLFPdm['prefix_templ'].isin(prefsd['prefix'][s])]
    dfLFPdm_sub_allLFP  = dfLFPdm_sub[['prefix_templ','bacc_all','bacc_all_shuffled']]
    dfLFPdm_sub_bestLFP = dfLFPdm_sub[['prefix_templ','bacc_best','bacc_best_shuffled']]

    dfLFPdm_sub_allLFP['prefix_templ'] = dfLFPdm_sub_allLFP['prefix_templ'].str.\
        replace('_subskip','_allLFP_subskip')

    dfLFPdm_sub_allLFP = dfLFPdm_sub_allLFP.rename(columns=\
                        {'bacc_all':'bacc', 'bacc_all_shuffled':'bacc_shuffled'})
    #dfLFPdm_sub_allLFP.drop( ['bacc_best','bacc_best_shuffled'],1)

    dfLFPdm_sub_bestLFP = dfLFPdm_sub_bestLFP.rename(columns=\
                        {'bacc_best':'bacc', 'bacc_best_shuffled':'bacc_shuffled'})
    #dfLFPdm_sub_bestLFP.drop( ['bacc_all','bacc_all_shuffled'], 1)

    dfLFPdm_sub_allLFP = dfLFPdm_sub_allLFP.reset_index()
    dfLFPdm_sub_bestLFP = dfLFPdm_sub_bestLFP.reset_index()

    dfc = pd.concat([dfLFPdm_sub_allLFP,dfLFPdm_sub_bestLFP])
    #dfc = dfc.drop('level_0',1)
    dfc = dfc.drop('index',1)
    #dfc

    df_allH = df_all[df_all['prefix_templ'].isin( prefsd['prefix_short'][s]) ]\
        [['prefix_templ','bacc', 'bacc_shuffled']]
    df_allmod = df_allH.groupby('prefix_templ').mean().reset_index()
    df_allmod['LFP channels'] = 'best'
    
    def lbd(row): 
        if row['prefix_templ'].find('allLFP') < 0:
            return 'all' 
        else:
            return 'best'
    #dfc2['LFP channels'] = None
    dfc['LFP channels'] = dfc.apply(lbd,1)
    
    dfc2 = pd.concat([dfc,df_allmod]).reset_index().drop('index',1)
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace(end,'', regex=0)
    dfc2['modality'] = dfc2['prefix_templ'].apply(lambda x: 'LFP + cortex' \
                                                  if x.find('modLFP') < 0 else 'LFP')
    
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace('modLFP_','')
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace('_allLFP','')

    dfc2['prefix_templ'] = dfc2['prefix_templ'].replace({'onlyH_act':'Hjorth activity',
                                                        'onlyH':'all Hjorth parameters'})

    dfc2['bacc'] = dfc2['bacc'].apply(lambda x : f'{x:.0f}%')
    dfc2['bacc_shuffled'] = dfc2['bacc_shuffled'].apply(lambda x : f'{x:.0f}%')


    rename_dict = {}
    rename_dict['prefix_templ'] = 'Feature set'
    rename_dict['modality'] = 'Data type'
    #rename_dict['interval_set'] = 'behav. states used'
    rename_dict['bacc'] = 'Performance'
    rename_dict['bacc_shuffled'] = 'Performance shuffled'
    #rename_dict['bacc_std'] = 'performance std'
    dfc2.rename(columns=rename_dict, inplace = 1)
    
    dfc2['Brain side'] = s

    dfc2r = dfc2[['Data type', 'LFP channels', 'Brain side', 'Feature set', 'Performance', 'Performance shuffled']]
    display(dfc2r)
    
    finals += [dfc2r]

In [ ]:
dfc

In [ ]:
df_allmod

In [ ]:
pd.concat(finals)

In [ ]:
'onlyH_act_subskip8^^'.replace(end,'')

In [ ]:
end

In [ ]:
dfc2['Feature set'].str.replace(end,'')

## plot table info (many bars)

In [ ]:
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes, crop_rawname=slice(None,None)) 
    plt.close()
    import gc;gc.collect()
    #break
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

## load labels (not important where from exactly)
from utils import loadLabelsDict
labels_dict = loadLabelsDict()

In [ ]:
subdir

# Perf only [old]

In [ ]:
perf_tuple = perf_to_use_list[-1]
defsp = 'special:min(sens,spec)'
score = 'bacc'

#TODO:  add part of plotCalcOutput
    #plt.title(plt.title())
scstr = ''
if score != defsp:
    scstr = score
fn_full = pjoin(gv.dir_fig,subdir,
                f'bars_perf_dif_subsets_{plotname_pref}_{scstr}.pdf')
plt.savefig(fn_full)

In [ ]:
prefixes

### H_act only

In [ ]:
from utils_postprocess_HPC import plotOnePrefQuick
pref_quick_plots = ['onlyH_act_subskip8BB']
for pref in pref_quick_plots:
    plotOnePrefQuick(rawnames, table_info_per_perf_type,perf_to_use_list[0], pref)

    plt.tight_layout()
    plt.savefig( pjoin(gv.dir_fig, subdir,f'{pref}_summary.pdf') )
    plt.close()

In [ ]:
from utils_postprocess_HPC import plotOnePrefQuick
pref_quick_plots = ['onlyH_act_modLFP_subskip8%%', 'onlyH_act_modLFP_subskip8^^', 
                      'onlyH_modLFP_subskip8%%', 'onlyH_modLFP_subskip8^^', 'onlyH_subskip8%%', 'onlyH_subskip8^^', 'onlyH_subskip8BB']
for pref in pref_quick_plots:
    plotOnePrefQuick(rawnames, table_info_per_perf_type,perf_to_use_list[0], pref)

    plt.tight_layout()
    plt.savefig( pjoin(gv.dir_fig, subdir,f'{pref}_summary.pdf') )
    plt.close()

In [ ]:
a = np.sort( np.unique( ords[1][ (ords[0]  == 'S07_on') & (ords[2]  == 'merge_nothing') ] ) )
#[aa for aa in a if aa.startswith()]
a

# Confmats and worst/best disnig states 

In [ ]:
#'onlyH_subskip8' in prefixes

In [ ]:
# 'onlyH_subskip8',
#  'onlyH_subskip8BB',
#  'onlyH_subskip8BL',
#  'onlyH_subskip8BR',
#  'onlyH_subskip8LL',
#  'onlyH_subskip8RR']

## Fig 1 -- 'onlyH_act_subskip8%%'

In [ ]:
#df_all.query('prefix_templ == "onlyH_modLFP_subskip8%%"')

In [ ]:
pref_confmat_plots = ['onlyH_modLFP_subskip8%%', 'onlyH_subskip8%%', 'onlyH_subskip8^^', 'onlyH_subskip8BB',
                      'onlyH_act_modLFP_subskip8%%', 'onlyH_act_modLFP_subskip8^^', 
                      'onlyH_modLFP_subskip8%%', 'onlyH_modLFP_subskip8^^', 
                      'onlyH_act_subskip8%%', 'onlyH_act_subskip8^^', 'onlyH_act_subskip8BB',
                     'LFPrel_noself_subskip8','LFPrel_noself_subskip8%%']

In [ ]:
pref_confmat_plots = ['onlyH_act_modLFP_subskip8@B-B', 'onlyH_act_subskip8@B-B']

In [ ]:
from utils_postprocess_HPC import filterOutputs, plotConfmats2, multiLevelDict2TupleList
import pandas as pd
from globvars import gp

### per medcond -- no plots, only stats

In [ ]:
pref_confmat_plots

In [ ]:
colnames_add = []
for it_mvt in ['hold', 'move','trem']:
    colnames_add += [it_mvt + '_perf_mean']
    colnames_add += [it_mvt + '_perf_std']

#%debug
    
dfcm = pd.DataFrame(columns = ['prefix_templ', 'grouping','medcond','interval_set','confmats','confmats_offdiag',
                              'confmats_diag'] + colnames_add)
notfound = []
mcpref2subdf = {}
grp,it = ('merge_nothing','basic')
for pref_confmat_plot in pref_confmat_plots:
    for medcond in ['on','off']:
        print(pref_confmat_plot)
        #grps = ['merge_nothing'] #, 'merge_movements']
        #grps = [ 'merge_movements']
#         outputs_filtered = postp.filterOutputs(output_per_raw,prefs=[pref_confmat_plot], 
#                                                grps=[grp], its=[it])
        dfc = df_all.query('prefix_templ in @pref_confmat_plot and grouping == @grp and '
                          'interval_set == @it and medcond == @medcond').reset_index()
        # works only if we have just one grp,it pair
        mcpref2subdf[(pref_confmat_plot,medcond)] = dfc
        #assert len(dfc)
        if len(dfc) == 0:
            notfound += [  (pref_confmat_plot, grp,it)]
            continue
        assert len(dfc) in [5,6]
        plt.rcParams.update({'font.size': 15})
        plt.rc('ytick',labelsize=22)
        plt.rc('xtick',labelsize=22)
        plt.rc('axes',labelsize=24)

        colorbar_axes_bbox = [0.80, 0.2, 0.025, 0.7]
        import sys, traceback
        #try:
            # note that it does not just extract confmats, it also resorts entries
        _,_,confmats_normalized,confmat_normalized_offdiags, confmat_normalized_diags_els = \
            plotConfmats2(dfc, output_per_raw, ww = 5, hh =5, keep_beh_state_sides=0,
                           keep_subj_list_title=1,
                           labelpad_cbar=155, colorbar_axes_bbox= colorbar_axes_bbox,
                          rename_class_names = {'notrem':'quiet'})
#         except Exception as e:
#             exc_info = sys.exc_info()
#             exc = traceback.TracebackException(*exc_info, capture_locals=True)
#             stackframe = exc_info[2].tb_next.tb_frame
#             local_vars_in_fun = stackframe.f_locals
#             #display(exc_info, stackframe,  exc_info[2].tb_next.tb_lineno)
#             #display(local_vars_in_fun)
#             raise e



        d = {'confmats':confmats_normalized, 'confmats_offdiag':confmat_normalized_offdiags,
            'confmats_diag': confmat_normalized_diags_els}
        d['grouping'] = grp
        d['medcond'] = medcond
        d['interval_set'] = it
        d['prefix_templ'] = pref_confmat_plot
        #plt.gcf().axes[-1].set_visible(False)
#             figname = f'confmats_{pref_confmat_plot}_{grp},{it}.pdf'
#             figname_full = pjoin(gv.dir_fig,subdir,figname)
#             plt.savefig(figname_full)
#             print(figname_full)
        plt.close()

        confmats_all = np.array( confmats_normalized )
        #confmats_all.shape
        cmm = confmats_all.mean(axis=0)
        cmstd = confmats_all.std(axis=0)        
        # diag el
        for it_mvt in ['notrem', 'hold', 'move','trem']:
            i = gp.int_types_basic.index(it_mvt)    
            me = cmm[i,i]
            std = cmstd[i,i]
            d[it_mvt + '_perf_mean'] = me
            d[it_mvt + '_perf_std'] = std

        # this is averaged over subjects/medcond 
        dfcm = dfcm.append(d,ignore_index = 1)
    print('notfound =',notfound)
dfcm_medcondsep = dfcm.reset_index() 
display(medcond, dfcm_medcondsep)

In [ ]:
dfcm_medcondsep[['prefix_templ','medcond', 'notrem_perf_mean', 'notrem_perf_std',
      'trem_perf_mean','trem_perf_std' ,
     'hold_perf_mean', 'hold_perf_std' ,  'move_perf_mean', 'move_perf_std']]

### per subject -- plots

In [ ]:
colnames_add = []
for it_mvt in ['hold', 'move','trem']:
    colnames_add += [it_mvt + '_perf_mean']
    colnames_add += [it_mvt + '_perf_std']

#%debug
    
dfcm0 = pd.DataFrame(columns = ['prefix_templ', 'grouping','interval_set','confmats','confmats_offdiag',
                              'confmats_diag'] + colnames_add)
notfound = []
for pref_confmat_plot in pref_confmat_plots:
    for grp,it in [('merge_nothing','basic')]: #, 'merge_movements']:
        print(pref_confmat_plot)
        #grps = ['merge_nothing'] #, 'merge_movements']
        #grps = [ 'merge_movements']
#         outputs_filtered = postp.filterOutputs(output_per_raw,prefs=[pref_confmat_plot], 
#                                                grps=[grp], its=[it])
        dfc = df_all.query('prefix_templ in @pref_confmat_plot and grouping == @grp and '
                          'interval_set == @it').reset_index()
        #assert len(dfc)
        if len(dfc) == 0:
            notfound += [  (pref_confmat_plot, grp,it)]
            continue
        assert len(dfc) == 11
        plt.rcParams.update({'font.size': 15})
        plt.rc('ytick',labelsize=22)
        plt.rc('xtick',labelsize=22)
        plt.rc('axes',labelsize=24)

        colorbar_axes_bbox = [0.80, 0.2, 0.025, 0.7]
        import sys, traceback
        #try:
            # note that it does not just extract confmats, it also resorts entries
        cax,_,confmats_normalized,confmat_normalized_offdiags, confmat_normalized_diags_els = \
            plotConfmats2(dfc, output_per_raw, ww = 5, hh =5, keep_beh_state_sides=0,
                           keep_subj_list_title=1,
                           labelpad_cbar=155, colorbar_axes_bbox= colorbar_axes_bbox,
                          rename_class_names = {'notrem':'quiet'})
#         except Exception as e:
#             exc_info = sys.exc_info()
#             exc = traceback.TracebackException(*exc_info, capture_locals=True)
#             stackframe = exc_info[2].tb_next.tb_frame
#             local_vars_in_fun = stackframe.f_locals
#             #display(exc_info, stackframe,  exc_info[2].tb_next.tb_lineno)
#             #display(local_vars_in_fun)
#             raise e
        cax.set_ylabel('% correct')
        
        
        d = {'confmats':confmats_normalized, 'confmats_offdiag':confmat_normalized_offdiags,
            'confmats_diag': confmat_normalized_diags_els}
        d['grouping'] = grp
        d['interval_set'] = it
        d['prefix_templ'] = pref_confmat_plot
        #plt.gcf().axes[-1].set_visible(False)
        figname = f'confmats_{pref_confmat_plot}_{grp},{it}.pdf'
        figname_full = pjoin(gv.dir_fig,subdir,figname)
        plt.savefig(figname_full)
        print(figname_full)
        plt.close()
        
        confmats_all = np.array( confmats_normalized )
        #confmats_all.shape
        cmm = confmats_all.mean(axis=0)
        cmstd = confmats_all.std(axis=0)        
        # diag el
        for it_mvt in ['hold', 'move','trem']:
            i = gp.int_types_basic.index(it_mvt)    
            me = cmm[i,i]
            std = cmstd[i,i]
            d[it_mvt + '_perf_mean'] = me
            d[it_mvt + '_perf_std'] = std
        
        # this is averaged over subjects/medcond 
        dfcm0 = dfcm0.append(d,ignore_index = 1)
print('notfound =',notfound)
dfcm0.reset_index()

In [ ]:
dfcm[['prefix_templ','grouping','interval_set','trem_perf_mean','trem_perf_std']]

## Prepare funtions (make human readalbe and wors tbest)

##### maybe old

In [ ]:
def getRegularPrefix(row, rawname, dforig):
    cols = ['prefix_templ']
    c = dforig['rawname'] == rawname
    for col in cols:
        c &= dforig[col] == row[col]
    r = dforig[c]
    assert len(r) == 1
    r = r.reset_index()
    r = r.loc[0]
    return r['prefix']
    #dforig.query('row')
getRegularPrefix(dfcm.reset_index().loc[0], 'S01_on', dforig)

In [ ]:
from utils_postprocess_HPC import prefTempl2pref, prefixTempl2Prefix

def getMcf(row, dforig):
    
    if 'rawname' not in row:
        #rawnames_cur = rawnames
        rawnames_cur = dforig['rawname'].unique()
    else:
        rawname = row['rawname']
        rawnames_cur = [rawname]
    res = []
    for rawname in rawnames_cur:
        if 'prefix' in row:
            prefix = row['prefix']
        else:
            prefix = getRegularPrefix(row, rawname, dforig)
        r = output_per_raw[rawname][prefix][row['grouping']][row['interval_set']]
        res += [r]
    return res
        
def getShuffledConfmatsStats(row, dforig):
    r = getMcf(row, dforig)

    cms = []
    for mcf in r:
        cm,lblord = getConfmat(mcf,row['grouping'],row['interval_set'],shuffled=1)
        cms += [cm]
    cms = np.array(cms)
    cms_mn = cms.mean(axis=0)
    cms_std = cms.std(axis=0)
    return cms_mn,cms_std
    
#row = dfcm_sub.reset_index().loc[0]
#cms_mn,cms_std = getShuffledConfmatsStats(row,dforig)


def getCM(row, data_type, stat, dforig):
    if data_type == 'normal':
        confmats_normalized =  row['confmats']
        confmats_all = np.array(confmats_normalized)
        cms_mn = confmats_all.mean(axis=0)
        cms_std = confmats_all.std(axis=0)
    else:
        cms_mn,cms_std = getShuffledConfmatsStats(row, dforig)
            
    if stat == 'mean':
        cms = cms_mn
    elif stat == 'std':
        cms = cms_std
    return cms

def getStat(row, data_type, stat, it_mvt, dforig):
    cms = getCM(row, data_type, stat, dforig)    
    i = gp.int_types_basic.index(it_mvt) 
    return cms[i,i]
    
def getBest(row, data_type, best_type, dforig, stat=None):
    cms = getCM(row, data_type, 'mean', dforig)
    assert cms.ndim == 2
    
    eye = np.eye(cms.shape[0])
    diag = np.diag(np.diag(cms))
    cms_offdiag = cms - diag
    cms_offdiag_largedval = cms_offdiag + eye * 1e5
    
    if best_type == 'best':
        ip = np.unravel_index( np.argmin( np.mean( cms_offdiag_largedval, 0)  ), cmm.shape )    
    elif best_type == 'worst':
        ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
    
    if stat is None:
        s = f'{gp.int_types_basic[ip[0]]} pred as {gp.int_types_basic[ip[1]]}'
        return s
    if stat == 'mean':
        return cms[tuple(ip)]
    elif stat == 'std':
        cms_std = getCM(row, data_type, 'std')
        return cms_std[tuple(ip)]
    # f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

### fill confusion matrix data for pub (compute best and worst)

In [ ]:
# from utils_postprocess_HPC import getConfmat
# cms = []
# #%debug
# for mcf in r:
#     cm,lblord = getConfmat(mcf,row['grouping'],row['interval_set'],shuffled=1)
#     cms += [cm]
# cms = np.array(cms)
# cms_mn = cms.mean(axis=0)
# cms_std = cms.std(axis=0)
# cms_mn,cms_std

In [ ]:
def getCM(row, data_type, stat, dforig):
    if data_type == 'normal':
        confmats_normalized =  row['confmats']
        confmats_all = np.array(confmats_normalized)
        cms_mn = confmats_all.mean(axis=0)
        cms_std = confmats_all.std(axis=0)
    else:
        cms_mn,cms_std = getShuffledConfmatsStats(row, dforig)
            
    if stat == 'mean':
        cms = cms_mn
    elif stat == 'std':
        cms = cms_std
    return cms

def getStat(row, data_type, stat, it_mvt, dforig):
    cms = getCM(row, data_type, stat, dforig)    
    i = gp.int_types_basic.index(it_mvt) 
    return cms[i,i]
    
def getBest(row, data_type, best_type, dforig, stat=None):
    cms = getCM(row, data_type, 'mean', dforig)
    assert cms.ndim == 2
    
    eye = np.eye(cms.shape[0])
    diag = np.diag(np.diag(cms))
    cms_offdiag = cms - diag
    cms_offdiag_largedval = cms_offdiag + eye * 1e5
    
    if best_type == 'best':
        ip = np.unravel_index( np.argmin( np.mean( cms_offdiag_largedval, 0)  ), cmm.shape )    
    elif best_type == 'worst':
        ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
    
    if stat is None:
        s = f'{gp.int_types_basic[ip[0]]} pred as {gp.int_types_basic[ip[1]]}'
        return s
    if stat == 'mean':
        return cms[tuple(ip)]
    elif stat == 'std':
        cms_std = getCM(row, data_type, 'std', dforig)
        return cms_std[tuple(ip)]
    # f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

In [ ]:
pref_confmat_plot

In [ ]:
dfcm

In [ ]:
#%debug
# best and worst pairs
#mcpref2subdf[(pref_confmat_plot,medcond)]
for (pref_confmat_plot,medcond),dforig in mcpref2subdf.items():
    rawnames = list( output_per_raw.keys())
    c = (dfcm['prefix_templ'] == pref_confmat_plot) & (dfcm['medcond'] == medcond)
    for data_type in ['normal','shuffled']:
        for stat in ['mean','std']:
            for it_mvt in ['notrem', 'hold', 'move','trem']:
                fcur = lambda row : getStat(row, data_type, stat, it_mvt, dforig)
                res = dfcm[c].apply(fcur, axis=1)
                dfcm.loc[c,f'detperf_{data_type}_{it_mvt}_{stat}'] = res                        

        if data_type != 'shuffled':            
            for best_type in ['best', 'worst']:            
                fcur = lambda row : getBest(row, data_type, best_type=best_type, dforig=dforig, stat=None)
                res = dfcm[c].apply(fcur, axis=1)
                dfcm.loc[c,f'{best_type}_pair'] = res                        

                for stat in ['mean','std']:
                    fcur = lambda row : getBest(row, data_type, best_type=best_type, dforig=dforig, 
                                                stat=stat)
                    res = dfcm[c].apply(fcur, axis=1)
                    dfcm.loc[c, f'{best_type}_pair_{stat}'] = res                                        
            
dfcm

In [ ]:
cols = ['prefix_templ',
 'detperf_normal_hold_mean',
 'detperf_normal_move_mean',
 'detperf_normal_trem_mean',
 'detperf_normal_hold_std',
 'detperf_normal_move_std',
 'detperf_normal_trem_std',
 'detperf_shuffled_hold_mean',
 'detperf_shuffled_move_mean',
 'detperf_shuffled_trem_mean',
 'detperf_shuffled_hold_std',
 'detperf_shuffled_move_std',
 'detperf_shuffled_trem_std',
 'best_pair',
 'best_pair_mean',
 'best_pair_std',
 'worst_pair',
 'worst_pair_mean',
 'worst_pair_std']

cols1 = ['prefix_templ',
 'detperf_normal_hold_mean',
 'detperf_normal_move_mean',
 'detperf_normal_trem_mean',
 'detperf_normal_hold_std',
 'detperf_normal_move_std',
 'detperf_normal_trem_std',
 'detperf_shuffled_hold_mean',
 'detperf_shuffled_move_mean',
 'detperf_shuffled_trem_mean',
 'detperf_shuffled_hold_std',
 'detperf_shuffled_move_std',
 'detperf_shuffled_trem_std']

cols2 =  ['medcond','prefix_templ','best_pair',
 'best_pair_mean',
 'best_pair_std',
 'worst_pair',
 'worst_pair_mean',
 'worst_pair_std']

cols_nice = []
for col in cols:
    if col.find('detperf_normal') >= 0:
        col_nice = col.replace('detperf_normal_','')
    else:
        col_nice = col
    cols_nice += [col_nice]
rename_dict = dict( zip(cols,cols_nice) )

In [ ]:
list( dfcm_sub['hold_mean'] )


In [ ]:
from globvars import interval_names_pub
#interval_names_pub = {'notrem':'quet','move':'grasp','trem':'tremor','hold':'hold'}

In [ ]:
dfcm_sub

In [ ]:
# show dataframes for best/worst stats AND print human readable
for pref_templ_desired in ['onlyH_act_subskip8@B-B', 'onlyH_act_modLFP_subskip8@B-B']:
    for medcond in ['off', 'on']:
        dfcm_sub = dfcm.query('prefix_templ == @pref_templ_desired and medcond == @medcond')
        assert len(dfcm_sub) == 1, len(dfcm_sub)
        #dfcm_sub = dfcm_sub[cols]
        dfcm_sub = dfcm_sub[cols2]
        #dfcm_sub = dfcm_sub[cols1]
        dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
        display(dfcm_sub)

        s = ''

        bp  = list( dfcm_sub[f'best_pair'] )[0]
        me  = list( dfcm_sub[f'best_pair_mean'] )[0]
        std = list( dfcm_sub[f'best_pair_std'] )[0]

        for it in interval_names_pub:
            if bp.find(it) >= 0:
                bp = bp.replace(it,f'\'{interval_names_pub[it]}\'' )
        bp = bp.replace('pred as','predicted to be')
        s += f'Least often detected confusion: {bp} ('
        s += f'mean={me:.0f}%, std={std:.0f}%). '

        bp  = list( dfcm_sub[f'worst_pair'] )[0]
        me  = list( dfcm_sub[f'worst_pair_mean'] )[0]
        std = list( dfcm_sub[f'worst_pair_std'] )[0]

        for it in interval_names_pub:
            if bp.find(it) >= 0:
                bp = bp.replace(it,f'\'{interval_names_pub[it]}\'' )

        bp = bp.replace('pred as','predicted to be')
        s += f'Most often detected confusion: {bp} ('
        s += f'mean={me:.0f}%, std={std:.0f}%); '

        print(s)

In [ ]:
#moc['pars']['subskip_fit']

#### smth relevantfor past vers of the draft

In [ ]:
# show dataframes for main stats
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    #dfcm_sub = dfcm_sub[cols]
    #dfcm_sub = dfcm_sub[cols2]
    dfcm_sub = dfcm_sub[cols1]
    dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
    display(dfcm_sub)
    


In [ ]:
# print human readable
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    #dfcm_sub = dfcm_sub[cols]
    #dfcm_sub = dfcm_sub[cols2]
    dfcm_sub = dfcm_sub[cols1]
    #dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
    #display(dfcm_sub)
    
    row = dfcm_sub.reset_index().loc(0)    
    #for data_type in ['normal', 'shuffled']:
    print(f'  {data_type}')
    for it_mvt in ['trem', 'hold', 'move']:
        s = ''
        #i = gp.int_types_basic.index(it_mvt)    
        #me = cmm[i,i]
        #std = cmstd[i,i]
        data_type = 'normal'
        me  = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_mean'] )[0]
        std = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_std'] )[0]
        
        data_type = 'shuffled'
        me_s  = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_mean'] )[0]
        std_s = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_std'] )[0]
        s += (f'for \'{interval_names_pub[it_mvt]}\' movement the '
            'statistic of correct detection is ')
        s += f'mean={me:.0f}, std={std:.0f}; '
        s += ' (for shuffled data '
        s += f'mean={me_s:.0f}, std={std_s:.0f}); '
        print(s)

In [ ]:
dfcm_sub.rename(rename_dict, axis='columns')

In [ ]:
dfcm[dfcm['prefix_templ']==pref_templ_desired  ]

In [ ]:
# to put in Discussion
#For all Hjorth parameters together

### Worst and confused pairs (absolut perf)

In [ ]:
len(dfcm[dfcm['prefix_templ']==pref_templ_desired  ])

In [ ]:
for pref_templ_desired in ['onlyH_act_subskip8B-B', 'onlyH_act_modLFP_subskip8B-B']:
    for medcond in ['off', 'on']:
        print('   ' ,pref_templ_desired)
        dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
        assert len(dfcm_sub) == 1
        confmats_normalized =  dfcm_sub['confmats']
        confmats_normalized = list(confmats_normalized)[0]
        assert len(confmats_normalized) == 11

        row = dfcm_sub.reset_index().loc[0]
        cms_mn,cms_std = getShuffledConfmatsStats(row)

        from globvars import gp
        confmats_all = np.array( confmats_normalized )
        #confmats_all.shape
        cmm = confmats_all.mean(axis=0)
        cmstd = confmats_all.std(axis=0)

        eyes = np.array( [np.eye(confmats_all.shape[-1] ) for cm in confmats_normalized], dtype=bool)
        confmat_normalized_offdiags_largedval = confmat_normalized_offdiags + eyes * 1e5

        ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
        print(f'Most often wrong attribution -- {gp.int_types_basic[ip[0]]} is predicted to be {gp.int_types_basic[ip[1]]}, '
        f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)')

        ip = np.unravel_index( np.argmin( np.mean( confmat_normalized_offdiags_largedval, 0)  ), cmm.shape )
        print(f'Least often wrong attibution -- {gp.int_types_basic[ip[0]]} is predicted to be {gp.int_types_basic[ip[1]]}, '
         f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

        s = ''
        for it_mvt in ['hold', 'move','trem']:
            i = gp.int_types_basic.index(it_mvt)    
            me = cmm[i,i]
            std = cmstd[i,i]
            s += f'for \'{it_mvt}\' movement the statistic of correct detection is '
            s += f'mean={me:.0f}, std={std:.0f}; \n'


        print(s)

In [ ]:
#prefixes

# onlyBar (bars per subject)

In [ ]:
# this is on purpose that values of *_only have 8 at the end and of others don't, 
# because they are processed differently
base_pref_start_dict = {'onlyH_subskip':'onlyH_modLFP_subskip', 
                        'onlyH_act_subskip':'onlyH_act_modLFP_subskip',
'onlyH_act_LFPand_only':'onlyH_act_modLFP_subskip8', 
                        'onlyH_act_only':'onlyH_act_modLFP_subskip8'}

In [ ]:
from utils_postprocess_HPC import fillStatinfo, findBestParcelGroups

In [ ]:
from utils_postprocess_HPC import getCorrespModLFP, addImprovColDfAll
#row = df_all[df_all['prefix'].str.contains('_only')].iloc[2]
#row = df_all[df_all['prefix'] == 'LFPrel_noself_subskip8BB'].iloc[0]
#row = df_all[df_all['prefix'] == 'onlyH_act_subskip8BB'].iloc[0]
#print(row)
#getCorrespModLFP(row, df_all, base_pref_start_dict=base_pref_start_dict, base_pref_start=None)

#df_all[df_all['prefix'] == 'onlyH_act_modLFP_subskipLL']
# takes some time to complete
#%debug
addImprovColDfAll(df_all,in_place=True, base_pref_start_dict=base_pref_start_dict) 
df_all[['prefix_templ','corresp_LFP_prefix_templ','improv']]

In [ ]:
dfc = df_all
from utils_postprocess_HPC import addRunCorrespCols
# parcel group name_RC and parcel group name nice RC
addRunCorrespCols(df_all, output_per_raw)

### prepare to work with areas

In [ ]:
df_all_allsep = df_all[(df_all['grouping'] == 'merge_nothing') & (df_all['interval_set'] == 'basic')]

from utils_postprocess_HPC import fillStatinfo, findBestParcelGroups, addParsColumns
addParsColumns(df_all,output_per_raw)
#%debug
addImprovColDfAll(df_all_allsep, base_pref_start_dict=base_pref_start_dict)

dfstat = findBestParcelGroups(df_all_allsep)
#dfstat = findBestParcelGroups(df_all_prev)
#subdf3
# here we should not have None
r = dfstat
r[(r['prefix'] =='onlyH_act_only8') & (r['operation_col'] == 'improv')][['rawname','parcel_group_names']]

In [ ]:
gs = ','.join( df_all['grouping'].unique().astype(str) )

dfstat.to_pickle( pjoin(gv.data_dir,subdir,f'dfstat_parcel_groups_{gs}.pkl') )
df_all.to_pickle( pjoin(gv.data_dir,subdir,f'df_all_{gs}.pkl') )

In [ ]:
df_all_prev = pd.read_pickle(pjoin(gv.data_dir,subdir,'df_all.pkl')) # only merge nothing

In [ ]:
#df_all_prev = pd.read_pickle(pjoin(gv.data_dir,subdir,'df_all.pkl')) # only merge nothing

In [ ]:
df_all['grouping'].unique()

In [ ]:
df_all_prev['grouping'].unique()

##### T-test to compare sided performance

In [ ]:
dfstat_sub = dfstat[ (dfstat['bestname'].isin(barnames_allowed)) &( dfstat['grouping'] == 'merge_nothing') &\
       (dfstat['interval_set'] == 'basic') & (dfstat['operation_col'] == 'improv') ]

In [ ]:
#dfstat_sub.groupby(['bestname']).groups.keys()

In [ ]:
onlyH_act_LFPand_best_among_clmove onlyH_act_LFPand_best_among_ilmove

In [ ]:
cl = 'onlyH_act_LFPand_best_among_clmove'
il = 'onlyH_act_LFPand_best_among_ilmove'
cld = dfstat_sub[dfstat_sub['bestname'] == cl]['bacc']
ild = dfstat_sub[dfstat_sub['bestname'] == il]['bacc']
#dfstat_sub.groupby(['bestname'])
from scipy import stats
r = stats.ttest_ind(cld,ild)
r
f't(10)={r.statistic:.2f}%, p={r.pvalue:.2f}'

In [ ]:
cl = 'onlyH_act_only_best_among_clmove'
il = 'onlyH_act_only_best_among_ilmove'
cld = dfstat_sub[dfstat_sub['bestname'] == cl]['bacc']
ild = dfstat_sub[dfstat_sub['bestname'] == il]['bacc']
#dfstat_sub.groupby(['bestname'])
from scipy import stats
r = stats.ttest_ind(cld,ild)
r
f't(10)={r.statistic:.2f}%, p={r.pvalue:.2f}'

## prepare plotspecs

In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP contralat',
                     'onlyH_act_modLFP_subskip8^^':'LFP ipsilat',
                     'onlyH_act_modLFP_subskip8BB':'LFP bilat',
                     'onlyH_act_only_best_among_single-sided': 'best area SS',
                     'onlyH_act_LFPand_quasibest_among_single-sided': '*LFP + best area SS',
                     'onlyH_act_LFPand_best_among_single-sided': 'LFP + best area SS',
                     'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_quasibest_among_clmove': '*LFP + best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM',
                     'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS',
                     good_prefs_permod['msrc'][0]:'cortex',
                    'onlyH_act_subskip8%%':'LFP+cortex contralat',
                     'onlyH_act_subskip8^^':'LFP+cortex ipsilat',
                     'onlyH_act_subskip8BB':'LFP+cortex bilat'}
plotspecs['all'] = prefix2final_name, 4,8  # numbers are width and height
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP contralat',
                     'onlyH_act_modLFP_subskip8^^':'LFP ipsilat',
                     'onlyH_act_modLFP_subskip8BB':'LFP bilat'}
plotspecs['onlyBestLFP'] = prefix2final_name, 1.8,5
prefix2final_name = { 'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS' }
plotspecs['two-sided'] = prefix2final_name, 1.8,5
prefix2final_name = { 'onlyH_act_only_best_among_single-sided': 'best area SS',
                     'onlyH_act_LFPand_quasibest_among_single-sided': '*LFP + best area SS',
                     'onlyH_act_LFPand_best_among_single-sided': 'LFP + best area SS' }
plotspecs['single-sided'] = prefix2final_name, 1.8,5
prefix2final_name = {'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_quasibest_among_clmove': '*LFP + best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM' }
plotspecs['contralat'] = prefix2final_name, 1.8,5

In [ ]:
plotspecs = {} 
prefix2final_name = {'onlyH_act_modLFP_subskip8%%': 'LFP CLM',
                     'onlyH_act_modLFP_subskip8^^': 'LFP ILM',
                     'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM',
                     'onlyH_act_only_best_among_ilmove': 'best area ILM',
                     'onlyH_act_LFPand_best_among_ilmove': 'LFP + best area ILM' }
plotspecs['contr_vs_ipsi'] = prefix2final_name, 1.8,5   # just above Figure 3

In [ ]:
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',
         'onlyH_act_only_best_among_clmove': 'best area',
         'onlyH_act_LFPand_best_among_clmove': 'LFP+best area',
         'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',                     
         'onlyH_act_subskip8%%':'LFP+cortex',
         'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
plotspecs['contra_various'] = prefix2final_name, 2.8,6

In [ ]:
# Fig 3
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',
         'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',
         'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
         'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
#'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area (two-sided)',
plotspecs['contra_various_TSonly_wandwoLFP'] = prefix2final_name, 2.2,6

In [ ]:
len(confmats_normalized)

In [ ]:
# Fig 3 (not in manuscript) with worse as well
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',  
                     'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',                                         
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
                     'onlyH_act_only_worst_among_two-sided': 'worst area (two-sided)',                                         
                     'onlyH_act_LFPand_worst_among_two-sided': 'LFP+worst area (two-sided)',
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
#'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area (two-sided)',
plotspecs['contra_various_TSonly_wandwoLFP_inc_worst'] = prefix2final_name, 2.2,7

In [ ]:
#plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',  
                     'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',                                          
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
plotspecs['contra_various_TSonly'] = prefix2final_name, 2.2,6

In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = { 'onlyH_act_subskip8%%':'Hjorth act contralat',
                     'onlyH_act_subskip8^^':'Hjorth act ipsilat',
                     'onlyH_act_subskip8BB':'Hjorth act bilat',
                     'onlyH_subskip8%%':'Hjorth act,mob,compl contralat',
                     'onlyH_subskip8^^':'Hjorth act,mob,compl ipsilat',
                     'onlyH_subskip8BB':'Hjorth act,mob,compl bilat'}
plotspecs['act_vs_all'] = prefix2final_name, 2.1,6


In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = { 'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS' }
plotspecs['two-sided'] = prefix2final_name, 1.8,5


In [ ]:
plotspecs, len(table_info_per_perf_type)

## run plot gen

In [ ]:
templs = genBestPGTempls()

In [ ]:
df_all_allsep = df_all[(df_all['grouping'] == 'merge_nothing') \
                       & (df_all['interval_set'] == 'basic')].copy()
#for cn in ['subject', 'medcond']:
#    df_all_allsep[cn] = df_all_allsep[cn].apply( lambda x: x[0] if isinstance(x,list) else x  , 1)

### onlyBar 

In [ ]:
prefix2final_name

In [ ]:
from utils_postprocess_HPC import addBestParcelGroups,genBestPGTempls,getMocFromRow
from utils_postprocess_HPC import  sidelet2sideTempl, addImprovCol
#plotname_pref = 'noLFP'
plotname_pref = ''
add_pg  = False

#%debug

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    score = 'bacc'
    prefixes_final = list( prefix2final_name.keys() )
    for perf_tuple in table_info_per_perf_type:
        print(f'Starting plotTableInfos_onlyBar for {perf_tuple}')
        if add_pg:
            addBestParcelGroups(output_per_raw, table_info_per_perf_type,
                                perf_tuple, score , do_add=1, 
                            remove_redundant_quasibest = False)

        plt.rc('axes', titlesize=18)
        plt.rc('axes', labelsize=20)
        plt.rc('ytick', labelsize=16)
        plt.rc('xtick', labelsize=16)
        
        k = list( table_info_per_perf_type.keys() )[0]
        axs, df = postp.plotTableInfos_onlyBar(table_info_per_perf_type,
                                           perf_tuple=perf_tuple,
                              output_subdir=subdir,use_recalc_perf=False,
                              prefixes_sorted=prefixes_final, prefix2final_name=prefix2final_name,
                                     crop_rawname='no',
                                           score= score,
                                           rawnames=rawnames, per_medcond =1,
                                          expand_best = 1, red='shuffled',
                                           allow_missing_prefixes = 1, hh =hh ,ww =ww)
        #axs[0,0].set_xlabel('')
        axs[2,1].set_visible(False)
        #frame1.axes.get_yaxis().set_visible(False)
        df['subject'] = df['rawname'].apply(lambda x: x.split('_')[0]  )
        df['medcond'] = df['rawname'].apply(lambda x: x.split('_')[1]  )
        df['barname_unsided'] = df['barname'].apply(lambda x: x[:-2])
        
        addImprovCol(df, plotspecs[ list(plotspecs.keys())[0] ][0])


    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
    plt.close()


In [ ]:
# best ipsi vs best contra

In [ ]:
# df_all_wbc = df_all.copy()
# for tmpl in templs:
#     bestname = tmpl[1]
#     df_all_wbc = df_all_wbc.assign(**{bestname:""})
#     #print(bestname)

In [ ]:
df

### Contra vs ipsi performance improvmenets [prog gen sentences]

In [ ]:
df = df_all_allsep

In [ ]:
dfgrp = df.groupby('barname_pre').mean()
dfgrp = dfgrp.reset_index()
d = plotspecs['contr_vs_ipsi'][0]
rename_dict = d.copy()
del rename_dict['onlyH_act_modLFP_subskip8^^']
del rename_dict['onlyH_act_modLFP_subskip8%%']
dfgrp = dfgrp[ dfgrp['barname_pre'].isin(list(rename_dict.keys())) ]
dfgrp['barname_pre'] = dfgrp['barname_pre'].replace(rename_dict)
s = ''
for index,row in dfgrp.iterrows():
    name = row['barname_pre']
    imp = row['improv']
    s += f'{name} showed {imp:.0f}% of performance improvement with respect to STN on the same side of the brain; '
    s = s.replace(' CLM',', contralateral to the moving body side,')
    s = s.replace(' ILM',', contralateral to the moving body side,')
    
s

### Plot bars with improvements

In [ ]:
barnames_allowed = [k for k in list( prefix2final_name.keys() ) if not k.startswith('onlyH_act_modLFP_subskip8')]
barnames_allowed

#### attempt 1 (using data from plotTableInfos_onlyBar )

In [ ]:
plotspecs['for_imporvements'] = prefix2final_name, 2,5

from utils_postprocess_HPC import addBestParcelGroups,genBestPGTempls,getMocFromRow,plotTableInfos_onlyBarDf
df['subject'] = df['rawname'].apply(lambda x: x.split('_')[0]) 
#plotname_pref = 'noLFP'
plotname_pref = ''
add_pg  = True

#%debuga

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    score = 'bacc'
    prefixes_final = list( prefix2final_name.keys() )
    assert len(table_info_per_perf_type) == 1
    print(f'Starting plotTableInfos_onlyBarDf for single perf tuple (processed earlier)')

    plt.rc('axes', titlesize=18)
    plt.rc('axes', labelsize=20)
    plt.rc('ytick', labelsize=16)
    plt.rc('xtick', labelsize=16)
    
    axs = plotTableInfos_onlyBarDf(df, barnames_allowed,                                          
                          output_subdir=subdir,use_recalc_perf=False,
                          prefixes_sorted=prefixes_final, prefix2final_name=prefix2final_name,
                                 crop_rawname='no',
                                       col1='improv', col2='improv_shuffled',
                                       rawnames=rawnames, per_medcond =1,
                                      expand_best = 1, hh =hh ,ww =ww, xlim=(0,50))
    #axs[0,0].set_xlabel('')
    axs[2,1].set_visible(False)
    #frame1.axes.get_yaxis().set_visible(False)

    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
    #plt.close()


## FIgure 5: Gen bars per subject (using df directly)

In [ ]:
#dfstat[dfstat['bestname'] == 'onlyH_act_only_best_among_single-sided']['parcel_group_names']

In [ ]:
# in improv in dfstat ther eis not parcel group names

In [ ]:
# Fig 3
plotspecs = {}  # plot name 2 stuff
prefix2final_name = {#'onlyH_act_modLFP_subskip8%%':'LFP contralat',
                     #'onlyH_act_modLFP_subskip8^^':'LFP ipsilat',
                     #'onlyH_act_modLFP_subskip8BB':'LFP bilat',
                     'onlyH_act_subskip8@%_exCB-%':'LFP+cortex contra',
                     'onlyH_act_subskip8@^_exCB-^':'LFP+cortex ipsi',
                     'onlyH_act_subskip8@B-B':'LFP+cortex bilat',
                     'onlyH_act_only_best_among_single-sided': 'best area',
                     #'onlyH_act_LFPand_quasibest_among_single-sided': '*LFP + best area SS',
                     'onlyH_act_LFPand_best_among_single-sided': 'LFP + best area'}


In [ ]:
barnames_allowed = [k for k in list( prefix2final_name.keys() ) \
                    if not k.startswith('onlyH_act_modLFP_subskip8')]
barnames_allowed

In [ ]:
from utils_postprocess_HPC import findBestParcelGroups, addImprovColDfAll
from utils_postprocess_HPC import addRunCorrespCols


df_all_allsep = df_all[(df_all['grouping'] == 'merge_nothing') \
                       & (df_all['interval_set'] == 'basic')].copy()

display( base_pref_start_dict )
#list(sorted( df_all_allsep['prefix_t'].unique() ))

add_pg = False
if add_pg:
    # long
    if 'parcel_group_name_RC' not in df_all_allsep:
        addRunCorrespCols(df_all_allsep, output_per_raw)
    #addImprovCol(df, plotspecs[ list(plotspecs.keys())[0] ][0])
    addImprovColDfAll(df_all_allsep, inplace = True, 
                      score_shuffled = 'bacc_perm_0.9',
                      base_pref_start_dict=base_pref_start_dict )    
    dfstat = findBestParcelGroups(df_all_allsep)
    

In [ ]:
#percentalize if we did not before
for col in ['bacc','bacc_shuffled','improv', 'improv_shuffled']:
    for df_ in df_all_allsep, dfstat:
        if df_[col].max() <= 1:
            df_[col] = df_[col] * 100

import gc; gc.collect()

In [ ]:
base_pref_start_dict

[pt for pt in prefix_templs if (pt is not None and pt.find('act_modLFP') >= 0) ]

In [ ]:
rawnames = list(sorted( dfstat['rawname'].unique() ) )
print(  rawnames )

In [ ]:
#assert np.all( ~r[(r['prefix'] =='onlyH_act_only8') &\
#                  (r['operation_col'] == 'improv')]['parcel_group_names'].isnull() )



plotspecs['for_imporvements'] = prefix2final_name, 2,6  # numbers are height and width
print( barnames_allowed )  # gives ordering

dfstat['subject'] = dfstat['rawname'].apply(lambda x: x.split('_')[0])
df_all_allsep['subject'] = df_all_allsep['rawname'].apply(lambda x: x.split('_')[0]) 

from utils_postprocess_HPC import (addBestParcelGroups,genBestPGTempls,
                                   getMocFromRow,plotTableInfos_onlyBarDf2)
#plotname_pref = 'noLFP'
plotname_pref = 'best_impr'
score = 'bacc'
#%debug

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    #prefixes_final = list( prefix2final_name.keys() )
    assert len(table_info_per_perf_type) == 1
    print(f'Starting plotTableInfos_onlyBarDf for single perf tuple (processed earlier)')

    plt.rc('axes', titlesize=18)
    plt.rc('axes', labelsize=20)
    plt.rc('ytick', labelsize=16)
    plt.rc('xtick', labelsize=16)

    #val = 'bacc'; xlim = (0,100)
    val = 'improv'; xlim = (-10,70)
    dfbar, axs = plotTableInfos_onlyBarDf2(df_all_allsep, dfstat, 
                            barnames_pre = barnames_allowed,                                          
                          output_subdir=subdir,use_recalc_perf=False,
                         crop_rawname='no',
                           col1=val, col2=f'{val}_shuffled',
                           sorted_by=val,
                            prefix2final_name = prefix2final_name,
                           rawnames=rawnames, per_medcond =1,
                          expand_best = 1, hh =hh ,ww =ww, xlim=xlim,
                                   xlabel = 'balanced accuracy [%]\n improvement with\n respect to LFP alone')
    #axs[0,0].set_xlabel('')
    axs[2,1].set_visible(False)
    #frame1.axes.get_yaxis().set_visible(False)
    
    for ax in axs.flatten():
        ax.axvline(x=0, ls=':')
        ax.grid(axis='x')
        
        import matplotlib.ticker as plticker
        loc = plticker.MultipleLocator(base=20.) # this locator puts ticks at regular intervals
        ax.xaxis.set_major_locator(loc)

    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'{preferred_pptype}_bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
 #plt.close()


### Which area consistently boosts performance?

In [ ]:
dfstat['bestname']

In [ ]:
dfstat.columns

In [ ]:
dfstat['name_nice']

In [ ]:
for coln in [ 'onlyH_act_only_best_among_clmove', 'onlyH_act_LFPand_best_among_clmove',]:
    print(coln)
    df_curbest = dfstat[dfstat['bestname'] == coln]
    #df_curbest[['subject','medcond','barname']]
    #.groupby('subject').count()
    #areas = df_curbest['barname_unsided']
    areas = df_curbest['name_nice']
    #display(areas)
    from collections import Counter
    a = dict(Counter(  areas  ))
    display(a)

    s = f'Among {len(areas)} datasets (5 subject ON and OFF and 1 subject OFF) '
    for area_name,c in a.items():
        s += f'{area_name} area was the best area {c} times, '
    #print(coln,'::   ',s,'\n')

In [ ]:
df1 = dfstat.query('bestname == "onlyH_act_LFPand_best_among_clmove" and operation_col == "bacc"').copy()
#\
    #[['rawname','bacc']]
df2 = df_all_allsep.query('prefix_templ == "onlyH_act_subskip8@B-B"').copy()

In [ ]:
df3 = pd.merge( df1.set_index('rawname'), df2.set_index('rawname'), on = 'rawname' )
df3

In [ ]:
d = df3['bacc_y'] - df3['bacc_x']
display(d)
d.mean(), d.std()

In [ ]:
# TODO wholectx - best

In [ ]:
preferred_pptype

In [ ]:
# TODO: adding beyond the best area improv

In [ ]:
#dfbar['improv_shuffled']

### Contra vs ipsi performance improvmenets

#### regen bars

In [ ]:
plotspecs = {} 
prefix2final_name = {'onlyH_act_modLFP_subskip8%%': 'LFP CLM',
                     'onlyH_act_modLFP_subskip8^^': 'LFP ILM',
                     'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM',
                     'onlyH_act_only_best_among_ilmove': 'best area ILM',
                     'onlyH_act_LFPand_best_among_ilmove': 'LFP + best area ILM' }
plotspecs['contr_vs_ipsi'] = prefix2final_name, 1.8,5   # just above Figure 3

In [ ]:
barnames_allowed = list( prefix2final_name.keys() )

In [ ]:
assert np.all( ~r[(r['prefix'] =='onlyH_act_only8') & (r['operation_col'] == 'improv')]['parcel_group_names'].isnull() )
print( barnames_allowed )  # gives ordering

add_pg  = False
if add_pg:
    dfstat = findBestParcelGroups(df_all)
dfstat['subject'] = dfstat['rawname'].apply(lambda x: x.split('_')[0])
df_all_allsep['subject'] = df_all_allsep['rawname'].apply(lambda x: x.split('_')[0]) 

from utils_postprocess_HPC import (addBestParcelGroups,genBestPGTempls,
                                   getMocFromRow,plotTableInfos_onlyBarDf2)
#plotname_pref = 'noLFP'
plotname_pref = 'best_impr'
#%debug

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    #prefixes_final = list( prefix2final_name.keys() )
    assert len(table_info_per_perf_type) == 1
    print(f'Starting plotTableInfos_onlyBarDf for single perf tuple (processed earlier)')

    plt.rc('axes', titlesize=18)
    plt.rc('axes', labelsize=20)
    plt.rc('ytick', labelsize=16)
    plt.rc('xtick', labelsize=16)

    val = 'bacc'; xlim = (0,100)
    #val = 'improv'; xlim = (-10,45)
    dfbar, axs = plotTableInfos_onlyBarDf2(df_all_allsep, dfstat, barnames_allowed,                                          
                          output_subdir=subdir,use_recalc_perf=False,
                         crop_rawname='no',
                           col1=val, col2=f'{val}_shuffled',
                           sorted_by=val,
                            prefix2final_name = prefix2final_name,
                           rawnames=rawnames, per_medcond =1,
                          expand_best = 1, hh =hh ,ww =ww, xlim=xlim,
                                   xlabel = 'balanced accuracy [%]\n improvement with\n respect to LFP alone')
    #axs[0,0].set_xlabel('')
    axs[2,1].set_visible(False)
    #frame1.axes.get_yaxis().set_visible(False)
    
    for ax in axs.flatten():
        ax.axvline(x=0, ls=':')

    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
 #plt.close()


In [ ]:
dfbar

#### gen sentences

In [ ]:
df = dfbar

dfgrp = df.groupby('barname_pre').mean()
dfgrp = dfgrp.reset_index()
d = plotspecs['contr_vs_ipsi'][0]
rename_dict = d.copy()
del rename_dict['onlyH_act_modLFP_subskip8^^']
del rename_dict['onlyH_act_modLFP_subskip8%%']
dfgrp = dfgrp[ dfgrp['barname_pre'].isin(list(rename_dict.keys())) ]
dfgrp['barname_pre'] = dfgrp['barname_pre'].replace(rename_dict)
s = ''
for index,row in dfgrp.iterrows():
    name = row['barname_pre']
    imp = row['improv']
    s += f'{name} showed {imp:.0f}% of performance improvement with respect to STN on the same side of the brain; '
    s = s.replace(' CLM',', contralateral to the moving body side,')
    s = s.replace(' ILM',', ipsilateral to the moving body side,')
    
s

In [ ]:
dfstat.columns

In [ ]:
set( dfstat['bestname'] )

In [ ]:
dfstat.columns[8:]

In [ ]:
df_all['prefix_templ']

In [ ]:
help(df_all_sub.sort_index)

In [ ]:
subdir

In [ ]:
#(r.index

In [ ]:
#r.count()

In [ ]:
### CLM / ILM
plotspecs = {} 
prefix2final_name = {#'onlyH_act_modLFP_subskip8%%': 'LFP CLM',
                     #'onlyH_act_modLFP_subskip8^^': 'LFP ILM',
                     'onlyH_act_only_best_among_clmove': 'best area',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area',
                     'onlyH_act_only_best_among_ilmove': 'best area',
                     'onlyH_act_LFPand_best_among_ilmove': 'LFP + best area' }
plotspecs['contr_vs_ipsi'] = prefix2final_name, 1.8,6.2

 # gives ordering
barnames_allowed = [
'onlyH_act_only_best_among_clmove',
'onlyH_act_LFPand_best_among_clmove',
'onlyH_act_only_best_among_ilmove',
 'onlyH_act_LFPand_best_among_ilmove']

print( barnames_allowed )  # gives ordering

add_pg  = False
if add_pg:
    dfstat = findBestParcelGroups(df_all)
dfstat['subject'] = dfstat['rawname'].apply(lambda x: x.split('_')[0])
df['subject'] = df['rawname'].apply(lambda x: x.split('_')[0]) 

from utils_postprocess_HPC import (addBestParcelGroups,genBestPGTempls,
                                   getMocFromRow,plotTableInfos_onlyBarDf2)
#plotname_pref = 'noLFP'
plotname_pref = 'contra_ipsi'
#%debug

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    #prefixes_final = list( prefix2final_name.keys() )
    assert len(table_info_per_perf_type) == 1
    print(f'Starting plotTableInfos_onlyBarDf for single perf tuple (processed earlier)')

    plt.rc('axes', titlesize=18)
    plt.rc('axes', labelsize=20)
    plt.rc('ytick', labelsize=16)
    plt.rc('xtick', labelsize=16)

    val = 'bacc'; xlim = (0,100)
    #val = 'improv'; xlim = (0,45)
    axs = plotTableInfos_onlyBarDf2(df_all, dfstat, barnames_allowed,                                          
                          output_subdir=subdir,use_recalc_perf=False,
                         crop_rawname='no',
                           col1=val, col2=f'{val}_shuffled',
                           sorted_by=val,
                            prefix2final_name = prefix2final_name,
                           rawnames=rawnames, per_medcond =1,
                          expand_best = 1, hh =hh ,ww =ww, xlim=xlim,
                                   xlabel = 'balanced accuracy [%]')
    #axs[0,0].set_xlabel('')
    axs[2,1].set_visible(False)
    #frame1.axes.get_yaxis().set_visible(False)

    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
 #plt.close()


#### to paste number (average improvement by best areas) in intro

In [ ]:
dftmp = dfstat [(dfstat['bestname'] == 'onlyH_act_LFPand_best_among_clmove') &
       (dfstat['operation_col'] == 'improv') ][ list(dfstat.columns[8:]) + ['improv'] ]


dftmp.mean(), dftmp.std()

# dfstat[ (dfstat['grouping'] == 'merge_nothing') &\
#        (dfstat['interval_set'] == 'basic') &\
#        #dfstat['prefix'].str.match('onlyH_act_LFPand.*') &\
#       dfstat['bestname'] == 'onlyH_act_LFPand_best_among_clmove'   ][['rawname','improv','name_nice']]

# df_all[~df_all['parcel_group_name_RC'].isnull()][ ['parcel_group_name_RC', 'parcel_group_names']]

## Print stats about parcel groups collected by _onlyBar

In [ ]:
list ( df['barname_pre_human'] )

In [ ]:
barnames = set( df['barname_pre_human'] ) # indep of subject names
print(barnames)
r = {}
rstd = {}
s = f'Performance statistics are: '
s_shuffled = f'Performance statistics for shuffled data are: '
for bn in barnames:
    dfsub = df[ df['barname_pre_human'] == bn ]
    me  = dfsub['p'].mean()
    std = dfsub['p'].std()
    r[bn] = me
    rstd[bn] = std
    s += f'{bn}: mean={me:.2f}%, std={std:.2f}%; '
    
    me  = dfsub['p_red'].mean()
    std = dfsub['p_red'].std()
    s_shuffled += f'{bn}: mean={me:.2f}%, std={std:.2f}%; '
r
s = s[:-2] + '.'
print(s)
print(s_shuffled)
display(r)

In [ ]:
s = 'shows that best STN LFP channel alone gives poor performance: '
bn = 'LFP'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 
##
s += 'Best сortical areas show higher performance than the best LFP channel: '
bn = 'best area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += 'Adding LFP features improves performance a bit further: '
bn = 'LFP+best area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += 'Using an entire entire cortex gives even better performance :'
bn = 'LFP+cortex (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += ('All feature subsets for every subject show performance '
'better than performance of the classifier trained on shuffled labels. ')
s += s_shuffled

s = s[:-2] + '.'
s

In [ ]:
s = 'Worst area performance had '
bn = 'worst area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. '

s += 'Adding LFP to it gave us performance with '
bn = 'LFP+worst area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. '

s = s[:-2] + '.'
print(s)

### Only one area [old]

In [ ]:
prefixes

In [ ]:
corresp

In [ ]:
#tpl[-1]['cmd']
 #[]

#### Filtering to leave one-sided and not-pgn-like

In [ ]:
tpll_new = []
basepref1 = 'onlyH_act_only'
basepref2 = 'onlyH_act_LFPand_only'
for tpl in tpll:
    prefix = tpl[1]
    g = 0
    d = dict( tpl[-1]['cmd'][0] )
    pgn = d.get('--parcel_group_names' ,None)
    if prefix.startswith(basepref1) or prefix.startswith(basepref2):        
        assert pgn is not None
        if pgn.endswith('_L') or pgn.endswith('_R'):
            tpll_new += [tpl]
            g = 1
    else:
        tpll_new += [tpl]
        g = 2
    #tpl[1]['cmd']
#     corresp,all_info = loadRunCorresp(tpl[-1])
#     ind,pgn2,nice_name = corresp.get(prefix, (None,None,None) )
#     if pgn2 is not None:
#         if pgn.endswith('_L') and pgn.endswith('_R'):
#             tpll_new += [tpl]
#         else:            
#             if pgn2 is not None:
#                 continue
    if g:
        print(g,prefix,pgn
    
#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)

In [ ]:
#tpll[0][:-1]

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup2,plotTableInfoBrain
from utils_postprocess_HPC import filterOutputs, tupleList2multiLevelDict
#%debug
prefs =  [pref for pref in prefixes if pref.startswith('onlyH_act_only') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_subskip8BB') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_modLFP_subskip8') ]
#prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_exclude') ]
#prefs += ['onlyH_act']
base_perf_prefix = 'onlyH_act_subskip8BB'
#base_perf_low_prefix = 'onlyH_act_modLFP_subskip8%%'
base_perf_low_prefix = None

#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)
outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=rawnames , prefs=prefs, grps = ['merge_nothing'] )
checkTupleListTableCompleteness(outputs_filtered)
#make_plots = False
make_plots = False
prefs_after_filter = {}
impr_per_rn = {}
infos = []
for rn in rawnames:
#for rn in ['S01_on', 'S02_on']:
    outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=[rn] , prefs=prefs, grps = ['merge_nothing'] )
    checkTupleListTableCompleteness(outputs_filtered)
    medcond = rn.split('_')[1]
    prefs_after_filter[rn] = list(outputs_filtered[rn].keys())
#    for mode in ['only','exclude']:
    for mode in ['only']:
        print(f'-------  Starting {rn} {medcond}  {mode}')
        impr_wrt_base_per_medcond_per_pgn=computeImprovementsPerParcelGroup2(outputs_filtered, 
                 base_perf_prefix = base_perf_prefix, base_perf_low_prefix = base_perf_low_prefix,
                 mode = mode, inv_exclude = True,
                 score = 'balanced_accuracy',
                 ignore_base_prefix_missing=1)
#         impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn,\
#         impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#             computeImprovementsPerParcelGroup(outputs_filtered, 
#                  'onlyH_act', mode = mode, inv_exclude = True,
#                  score = 'balanced_accuracy',
#                  ignore_base_prefix_missing=1)
#         impr_per_rn[rn] = impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn,\
#             impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond
        impr_per_rn[rn] = impr_wrt_base_per_medcond_per_pgn
    #        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond
        #break
        for medcond in impr_wrt_base_per_medcond_per_pgn.keys():
            if len(impr_wrt_base_per_medcond_per_pgn[medcond] ) <= 4:
                print(f'WARNING: {rn} {medcond} has zero len impr_wrt_base_per_medcond_per_pgn[{medcond}]')
                continue            
            axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                   head_subj_ind=None, mode=mode, subdir=subdir,
                    savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_',
                                               save_only = not make_plots)
            infos += [(mode,medcond,info)]
            if make_plots:
                plt.close()
            #break
        #break

In [ ]:
#impr_wrt_base_per_medcond_per_pgn

In [ ]:
set(prefs_after_filter['S01_on'] ) ^ set( prefs_after_filter['S02_on'] )

rn1 = 'S01_on'
outputs_filtered1 = postp.filterOutputs(output_per_raw, rns=[rn1] , prefs=prefs, grps = ['merge_nothing'] )
l1 = list(outputs_filtered1[rn1].keys())
rn2 = 'S02_on'
outputs_filtered2 = postp.filterOutputs(output_per_raw, rns=[rn2] , prefs=prefs, grps = ['merge_nothing'] )
l2 = list(outputs_filtered2[rn2].keys())
set(l1) ^ set(l2)

'2 ' *2

list(sorted( output_per_raw['S02_on'].keys() ) )

outputs_filtered2['S02_on']['onlyH_act_only15'].keys()

oo=0
tpll2 = pp.multiLevelDict2TupleList(outputs_filtered2,4,3)
tpll_reshaped2 = list( zip(*tpll2) )
oo=0
tpll1 = pp.multiLevelDict2TupleList(outputs_filtered1,4,3)
tpll_reshaped1 = list( zip(*tpll1) )

len( outputs_filtered2[rn2] ), len( outputs_filtered1[rn1] )

len( tpll_reshaped2[1] ), len( tpll_reshaped1[1] )

list ( zip(l1,l2) )

prefs_after_filter['S02_on']

list( outputs_filtered['S02_on'].keys() )

m = (tpll_reshaped[0] == 'S02_on') & (tpll_reshaped[2] == 'merge_nothing')
list ( tpll_reshaped[1][m] ) , list( tpll_reshaped[3][m] )

rn = 'S07_on'
aa = impr_per_rn[rn][rn.split('_')[-1]]

impr_per_rn

In [ ]:
order = ['LFP', 'Sensorimotor', 'Cerebellum',
    'FrontalMed',
 'TemporalMid',
 'SupraMarginal',
 'OccipitalInf',
 'Angular',
 'FrontalInf',
 'ParietalSup',
 'TemporalSup',
 'FrontalSup',
 'TemporalInf',
 'OccipitalSup',
 'OccipitalMid',
 'ParietalInf']
keyorder = lambda tpl: order.index( tpl[0] )

#### Make hor barplots one per subj, one bar per area, each medcond dif color

In [ ]:
subjs = list(sorted(set( [rn.split('_')[0] for rn in rawnames] ) ))
#dict_keys(['sens_recalc', 'spec_recalc', 'descr', 'comment_from_runstrings', 'sens', 'spec', 
# 'F1', 'acc', 'bacc', 'sens_red', 'spec_red', 'sens_red_recalc', 'spec_red_recalc', 'F1_red', 'num', 'num_red'])
aa = table_info_per_perf_type[perf_to_use_list[0]]
tpls = sorted( list(aa.keys() ), key=lambda x: x[0] )

nr = len(subjs)
nc = 1
ww = 5; hh = 6
fig,axs = plt.subplots(nr,nc,figsize=( nc*ww, nr*hh), sharex='col')
plt.subplots_adjust(top=0.97,bottom=0.02,left=0.25,right=0.99,hspace=0.1)

color_per_medcond = dict(ON='blue',OFF='red')

for rn,imprs in impr_per_rn.items():
    #rn,grp,it = tpl
    subj,medcond = rn.split('_')
    axi = subjs.index(subj)
    impr_cur = imprs[medcond]
    
    medcond = medcond.upper()
    srt = sorted(impr_cur.items(), key=keyorder)
    srt = list(srt)[::-1]
    ks,vs=  zip(*srt)
    
    ax = axs[axi]
    ax.barh(ks,vs, color=color_per_medcond[medcond], alpha=0.7 )
    
    print(rn,axi)
    print(rn,srt)
    
    #rpint(aa[tpl].keys() )
    #print( aa[tpl]['onlyH_act'].keys() )
    
    #val = aa[tpl]['onlyH_act']['bacc']
    #print(rn, axi, val )
    
    #ax.barh([medcond ],[val] )
    ax.set_xlim(0,100)
    ax.set_title(subj)
    ax.set_xlabel('balanced acc')
    #break
#.keys()
#plt.tight_layout()
plt.suptitle('Perf per brain area (red=OFF, blue=ON)', y=0.99)
plt.savefig( pjoin(gv.dir_fig, subdir,'H_act_perf_summary_per_area.pdf') )
plt.close()

In [ ]:
#impr_wrt_base_per_medcond_per_pgn

In [ ]:
axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                               head_subj_ind=None, mode=mode, subdir=subdir,
                                               savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_')

In [ ]:
impr_wrt_base_per_medcond_per_pgn

# Improvement compared to LFP

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup2,computeImprovementsPerParcelGroup,plotTableInfoBrain

base_perf_prefix = 'onlyH_act_subskip8BB'
base_perf_low_prefix = 'onlyH_act_modLFP_subskip8%%'


pref_LFPand = [pref for pref in prefixes if pref.startswith('onlyH_act_LFPand_') ]
pref_LFPand += [base_perf_prefix, base_perf_low_prefix]

#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)
make_plots = False

#%debug
ress = []
for rn in rawnames:
    outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=[rn], prefs=pref_LFPand, grps = ['merge_nothing'])
    medcond = rn.split('_')[1]
    #for mode in ['LFPand_only','LFPand_exclude']:
    for mode in ['LFPand_only']:
        print(f'-------  Starting {rn} {medcond}  {mode}')
#         impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#             computeImprovementsPerParcelGroup(outputs_filtered, 
#                   'onlyH_act', 'modLFP',mode = mode, inv_exclude = True,
#                    score='balanced_accuracy')
        impr_wrt_base_per_medcond_per_pgn=computeImprovementsPerParcelGroup2(outputs_filtered, 
                 base_perf_prefix = base_perf_prefix, base_perf_low_prefix = base_perf_low_prefix,
                 mode = mode, inv_exclude = True,
                 score = 'balanced_accuracy',
                 ignore_base_prefix_missing=1)
    #        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond

        for medcond in impr_wrt_base_per_medcond_per_pgn.keys():
            if len(impr_wrt_base_per_medcond_per_pgn[medcond] ) <= 5:
                print(f'WARNING: {rn} {medcond} has zero len impr_wrt_base_per_medcond_per_pgn[{medcond}]')
                continue
            axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                    head_subj_ind=None, mode=mode, subdir=subdir, 
                    savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_',
                    save_only = not make_plots)

            ress += [info]
            plt.close()
            #break
        #break
resgood = [a is not None for a in ress]
import gc; gc.collect()

In [ ]:
impr_wrt_base_per_medcond_per_pgn

In [ ]:
mode = 'LFPand_only'

In [ ]:
# impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#     computeImprovementsPerParcelGroup(outputs_filtered, 'onlyH_act', 'modLFP',
#                                       mode = mode, inv_exclude = True)

In [ ]:
impr_per_medcond_per_pgn

In [ ]:
pp.printDict(outputs_filtered,depth_cur=2)

In [ ]:
outputs_filtered.keys()

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
[ (ii,tpl[:-1], len(tpll[ii][-1]['feature_names_filtered']) ) for ii,tpl in enumerate(tpll ) ]

In [ ]:
ii = 25
print(tpll[ii][:-1])
len(tpll[ii][-1]['feature_names_filtered'])

In [ ]:
len(tpll[ii][-1]['featnames_for_fit'])

In [ ]:
tpll[ii][-1]['cmd']

In [ ]:
tpll[ii][-1].keys()

# Main feature signif plots

In [ ]:
prefixes

In [ ]:
import utils_postprocess as pp
from utils_postprocess_HPC import loadFullScores 
from matplotlib.lines import Line2D


#%debug
hh_ = None
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
#for prefix_cur in [ 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyCon']:
#for prefix_cur in [ 'allb_beta', 'onlyH']:
#for prefix_cur in [  'allb_beta', 'allb_tremor', 'allb_gamma', 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyRbcorr', 'LFPrel_noself_onlyCon', 'modSrc_self',  'onlyCBSrc', 'onlyMotorSrc']:
#for prefix_cur in [ 'LFPrel_noself_onlyBpcorr', 'LFPrel_noself']:
#for prefix_cur in [ 'all']:
#for tpl_ in [('onlyH',10), ('LFPrel_noself',25)]:
#for tpl_ in [('LFPrel_noself_onlyRbcorr',15)]:
#for tpl_ in [ ('allb_beta',15),  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [ ('LFPrel_noself_onlyBpcorr',25)]:    
#for prefix_cur in [ 'onlyH', 'LFPrel_noself', 'allb_beta']:
#for prefix_cur in prefixes:

pref_hh_tuples = [ ('all',40)]
pref_hh_tuples = [ ('onlyH',10)]
pref_hh_tuples = [('LFPrel_noself',35) ]

pref_hh_tuples = [ ('onlyH_act',40)]
feat_subset_name = 'all'

subdir_short = 'per_subj_per_medcond_best_LFP'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
#grpit_tpl = 'merge_movements','basic'

#subdir_short = 'joint2_noskip'

from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_HPC.py'
ipython.magic(runstr_)
#             break+
#         break
#     break


# EBM

In [ ]:
prefixes

In [ ]:
aa = [   'onlyH',
 'onlyH_act',
 'onlyH_act_CB',
 'onlyH_act_CByFS',
 'onlyH_act_CByFSyPS',
 'onlyH_act_CByFSyPSyPI',
 'onlyH_act_CByFSyPSyPIyFM',
 'onlyH_act_CBySM',
 'onlyH_act_noCB',
 'onlyH_act_noCBnoFS',
 'onlyH_act_noCBnoFSnoPS',
 'onlyH_act_noCBnoFSnoPSnoPI',
 'onlyH_act_noCBnoFSnoPSnoPInoFM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoA',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTS',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTSnoFI',
 'onlyH_act_noFrontalMed',
 'onlyH_act_noFrontalSup',
 'onlyH_act_noLFP',
 'onlyH_act_noOccipitalMid',
 'onlyH_act_noParietalSup',
 'onlyH_act_noSensorimotor',
 'onlyH_act_noTemporalMid',   ]

In [ ]:
aa = [ 
 'allb_beta_noH',
 'allb_gamma_noH',
 'allb_tremor_noH']

In [ ]:
[tpl[2:-1] for tpl in tpll]

In [ ]:
subdir = 'per_subj_per_medcond_best_LFP'
subdir_short = subdir

pref_hh_tuples = [ ('all',40)]
EBM_feat_subsets = ['VIFsel']

pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5), ('onlyH',20), ('onlyH_act',20), ('LFPrel_noself',40)]
pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5)]
pref_hh_tuples = [ ('LFPrel_noself',40) ]
# pref_hh_tuples = [ ('onlyH_act_noCB',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFS',20)]
# pref_hh_tuples =  [(a,10) for a in aa]
#pref_hh_tuples =  [(a,40) for a in aa]
##pref_hh_tuples = [ ('modSrc',50)]
pref_hh_tuples = [ ('onlyH_act',7)]
#pref_hh_tuples = [ ('modLFP',3)]
#pref_hh_tuples = [ ('onlyH',10)]
#onlyH_act_noCBnoFS
# pref_hh_tuples = [ ('onlyH_act_noFrontalSup',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFSnoPSnoPI',20)]

EBM_feat_subsets = ['all']


#subdir_short = 'joint_noskip'
#grpit_tpl = 'merge_movements','basic'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
#grpit_tpl = 'merge_all_not_trem','basic'

import gc; gc.collect()
#%debug
from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_EBM_HPC.py'
ipython.magic(runstr_)

In [ ]:
int( keystr[1:3] )

In [ ]:
outputs_grouped.keys()

In [ ]:
prefixes

In [ ]:
from datetime import datetime
fname = mult_clf_output['filename_full']
mtime = os.stat(  fname).st_mtime
datetime.fromtimestamp(mtime)

In [ ]:
fname

In [ ]:
mult_clf_output['cmd']

In [ ]:
mult_clf_output_clf_output_clf_output_clf_output     

In [ ]:
d = ['Cerebellum','ParietalSup','FrontalSup', 'ParietalInf','Angular', 'OccipitalInf', 'FrontalMed']
labels_dict['all_raw']

In [ ]:
clf_dict = mult_clf_output['featsel_per_method']['interpret_EBM']['all']
print(clf_dict.keys())

from interpret import preserve
explainer = clf_dict['explainer']
scores = clf_dict['scores']

In [ ]:
sc_noi, fns_noi, _,_ = postp.splitScoresEBM(scores,explainer.feature_names)

In [ ]:
sc_noi

In [ ]:
fn_to_observe = fns_noi[-1]
print(fn_to_observe)
preserve(explainer, fn_to_observe)

In [ ]:
mult_clf_output['feature_names_filtered']

In [ ]:
dd = 'bias/6.1694/2.4873'
np.sum( np.array(list(dd)) == '/' ) 


In [ ]:
import gc;gc.collect()

In [ ]:
common = set(mult_clf_output['VIF_truncation']['colinds_bad_VIFsel'])
#print(common)
for i,og in enumerate(outputs_grouped.items() ):
    rn = og[0][0]
    (prefix,grp,int_type), mult_clf_output = og[1]
    VIFtr = mult_clf_output['VIF_truncation']
    badinds = VIFtr['colinds_bad_VIFsel']
    print(len(badinds), badinds)
    common = common & set(badinds)
    
print(len(common),sorted(common,reverse=1) )

In [ ]:
d[(rn,prefix,grp,it)]

In [ ]:
VIFtr.keys()

In [ ]:
VIFtr['VIFsel_linreg_objs'][0].__dict__.keys()

In [ ]:
d = {}
for rn,prefix,grp,it,mult_clf_output in tpll:
    VIFtr = mult_clf_output['VIF_truncation']
    dd = dict( VIFtr.items() )
    #del dd['VIFsel_linreg_objs']
    #dd[]
    #VIFtr['VIFsel_linreg_objs'][0]
    d[(rn,prefix,grp,it)] = dd
#('S01,S02,S04,S05,S07_off', 'allb_beta', 'merge_movements', 'basic')
VIF_info_fname = pjoin(gv.data_dir,subdir_short,'VIF_info.npz')
print(VIF_info_fname)
np.savez(VIF_info_fname,VIF_info = d)

In [ ]:
# (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
#      scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]


runMLvars_pso = []
for cso in collect_SHAP_outs:
    (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = cso 
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    #assert (prefix,grp,int_type) == (prefix_,grp_,int_type_)    
    cmd = mult_clf_output['cmd']
    print(cmd[0])

    ####################

    s = '%run -i ../run/run_ML.py '
    for a,b in cmd[0]:
        s += f'{a} {b} '
    print(s)
    from IPython import get_ipython; ipython = get_ipython()

    #%run -i ../run/run_ML.py -r S01_on_hold,S01_on_move,S02_on_hold,S02_on_move,S04_on_hold,S04_on_move,S05_on_hold,S05_on_move,S07_on_hold,S07_on_move --param_file ML_joint_one_LFP_HPC.ini --groupings_to_use merge_movements --int_types_to_use basic --feat_types H_act,H_mob,H_compl --prefix onlyH --SLURM_job_id 114703_22 --calc_MI 0

    ipython.magic(s + ' --exit_after artif_processed --show_plots 0 --do_cleanup 0')
    anndict_per_intcat_per_rawn_out = anndict_per_intcat_per_rawn

    runMLvars = {}
    runMLvars['featnames_nice'] = featnames_nice
    runMLvars['featnames'] = featnames
    runMLvars['X_pri'] = X_pri
    runMLvars['rawnames'] = rawnames
    runMLvars['Xtimes_pri'] = Xtimes_pri
    runMLvars['wbd_pri'] = wbd_pri
    runMLvars['anndict_per_intcat_per_rawn'] = anndict_per_intcat_per_rawn
    runMLvars['new_main_side_pri'] = new_main_side_pri
    runMLvars['roi_labels'] = roi_labels
    runMLvars['srcgrouping_names_sorted'] = srcgrouping_names_sorted
    runMLvars['rawtimes_pri'] = rawtimes_pri
    
    runMLvars_pso += [runMLvars]

    rng = slice(0,10)
    
    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    featis = featis[rng] #np.random.randint( len(featnames),size=20)

    # anns, anns_pri, times_concat, dataset_bounds, wbd_merged = utsne.concatAnns(rawnames,
    #                                                           rawtimes_pri, crop=(crop_start,crop_end),
    #                                                           side_rev_pri = side_switch_happened_pri,
    #                                                          wbd_pri = wbd_pri, sfreq=sfreq, ret_wbd_merged=1)
    # print('times_concat end {} wbd end {}'.format(times_concat[-1] * sfreq, wbd_merged[1,-1] ) )

    # ivalis = utils.ann2ivalDict(anns)
    # ivalis_tb_indarrays_merged = \
    #     utils.getWindowIndicesFromIntervals(wbd_merged,ivalis,
    #                                         sfreq,ret_type='bins_contig',
    #                                         ret_indices_type = 'window_inds',
    #                                         nbins_total=nbins_total )

    # anndict_per_intcat = f['anndict_per_intcat'][()]
    # anndict_per_intcat_per_rawn[rawn] = anndict_per_intcat
    # bindict_per_rawn[rawn] = upre.markedIntervals2Bins(anndict_per_intcat,
    #                                                    rawtimes,sfreq,wbd=wbd)
    xlim = (-4,8)
    #xlim = None
    figname = 'hists_'+ f'{(rn_,prefix,grp,int_type,label_str,fsh)}_rng={rng}.pdf'
    print('start plotting ',figname)
    postp.plotFeatHists(rawnames,featnames_nice,featis,X_pri,Xconcat, bindict_per_rawn,
                      ivalis_tb_indarrays_merged, xlim_common = xlim,nbins=25, savefig = 0)
    
    
    plt.savefig(pjoin(gv.dir_fig, subdir_short, figname))
    plt.close()


In [ ]:
import gc; gc.collect()

In [ ]:
(prefix_,grp_,int_type_)

## get info from run_genfeats (for a fixed rn_)

In [ ]:
mainLFPchan_per_rawn = {}
for runMLvars in runMLvars_pso:
    for rawn in runMLvars['rawnames']:
        subj,medcond,task  = utils.getParamsFromRawname(rawn)
        mainLFPchan_cur = best_LFP_info[subj][f'{best_LFP_prefix},merge_movements,basic']['best_LFP']
        #print(rawn,subj,mainLFPchan_cur)
        mainLFPchan_per_rawn[rawn] = mainLFPchan_cur

In [ ]:



rns = list(output_per_raw.keys() )
genfeat_info_per_rn = {}
for rn_to_plot in rns:
    mult_clf_output = output_per_raw[rn_][prefix_][grp][int_type]
    ML_info = mult_clf_output['info']
    fn_feat = ML_info['fname_feat_full_pri']

    dat_out_pri = []
    dat_lfp_hires_out_pri = []
    times_hires_out_pri = []
    times_out_pri = []
    subfeature_order_out_pri = []
    subfeature_order_lfp_hires_out_pri = []
    extdat_out_pri = []
    rawnames_out = []
    emgdat_pri = []
    
    for i,fn_feat_cur in enumerate(fn_feat):
        ffeat = np.load(fn_feat_cur, allow_pickle=True )
        ss = ' '.join([' '.join(list(tpl)) for tpl in  ffeat['cmd'][0]] )
        s = '%run -i ../run/run_genfeats.py ' + ss + ' --exit_after prescale_data'
        print(s)
        ipython.magic(s)

        dat_out_pri              += dat_pri
        dat_lfp_hires_out_pri    += dat_lfp_hires_pri
        times_hires_out_pri      += times_hires_pri
        times_out_pri            += times_pri
        subfeature_order_out_pri += subfeature_order_pri
        subfeature_order_lfp_hires_out_pri += subfeature_order_lfp_hires_pri
        extdat_out_pri           += extdat_pri
    
    import re
    for fn in fn_feat:
        r = re.match('.*(S[0-9]+_[a-z]+_[a-z]+)_.*',fn).groups()[0]
        #rint(r)
        rawnames_out += [r]


    emgraw_pri = []
    for rawi,rawn in enumerate(ML_info['rawnames']):
        emgraw = mne.io.read_raw_fif ( pjoin(gv.data_dir,rawn + '_emg_rectconv.fif') )
        chns_cur = gv.EMG_per_hand[main_side_pri[rawi]]
        emgraw.pick_channels(chns_cur)
        emgraw.resample(256)
        #emgraw_pri += [emgraw]
        emgdat_pri += [emgraw.get_data()]
        assert emgdat_pri[-1].shape[1] == len(times_out_pri[rawi])

    from featlist import replaceMEGsrcChnamesParams
    subfeature_order_newsrcgrp_pri = ML_info.get('chnames_newsrcgrp_pri')
    if subfeature_order_newsrcgrp_pri is None:
        subfeature_order_newsrcgrp_pri = [0] * len(rawnames_out)
        for rawi in range(len(runMLvars['rawnames'])):
            subfeature_order_newsrcgrp_pri[rawi] = \
                replaceMEGsrcChnamesParams(subfeature_order_out_pri[rawi], 0,9, '.*', 0)
            
            
    d = {}
    d['dat_out_pri'] = dat_out_pri
    d['dat_lfp_hires_out_pri'] = dat_lfp_hires_out_pri
    d['times_hires_out_pri'] = times_hires_out_pri
    d['times_out_pri'] = times_out_pri
    d['subfeature_order_out_pri'] = subfeature_order_out_pri
    d['subfeature_order_lfp_hires_out_pri'] = subfeature_order_lfp_hires_out_pri
    d['extdat_out_pri'] = extdat_out_pri
    d['rawnames_out'] = rawnames_out
    d['emgdat_pri'] = emgdat_pri
    d['subfeature_order_newsrcgrp_pri'] = subfeature_order_newsrcgrp_pri
    genfeat_info_per_rn[rn_to_plot] = d

# best_LFP_info_fname = pjoin(gv.data_dir, 'best_LFP_info.json')
# with open(best_LFP_info_fname, 'r') as f:
#     best_LFP_info = json.load(f)



display(mainLFPchan_per_rawn)

In [ ]:
rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats

In [ ]:
# plot time traces

for collect_SHAP_outs_ind in range(len(collect_SHAP_outs)):
    #collect_SHAP_outs_ind = 0
    (rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[collect_SHAP_outs_ind]
    print(rn_)
    
    runMLvars = runMLvars_pso[collect_SHAP_outs_ind]
    d = genfeat_info_per_rn[rn_]
    
#     ##########################  just for now to avoid recalc of genfeats
#     if rn_.endswith('_off'):
#         continue
#     ###########################    
        
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    VIF_truncation = mult_clf_output['VIF_truncation']
    colinds_good_VIFsel = VIF_truncation['colinds_good_VIFsel']
    featnames_VIFsel = runMLvars['featnames'][colinds_good_VIFsel]
    featnames_best_LFP = runMLvars['featnames']

    from globvars import gp
    beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    #%debug
    rng = slice(0,8)
    #rng = slice(0,2)
    rng_feats = slice(0,2)
    #rng_feats = slice(0,1)
    from plots import plotFeatsAndRelDat
    
    axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                             featnames_VIFsel[featis[rng_feats]], d['dat_out_pri'],
                       d['subfeature_order_out_pri'],
                    runMLvars['X_pri'],[featnames_best_LFP]*len(d['rawnames_out']),
                    d['times_out_pri'],
                    runMLvars['Xtimes_pri'],
                    d['subfeature_order_newsrcgrp_pri'], runMLvars['wbd_pri'],
                    dat_hires_pri=d['dat_lfp_hires_out_pri'],
                    chnames_all_hires_pri = d['subfeature_order_lfp_hires_out_pri'],
                    times_hires_pri=d['times_hires_out_pri'],
                    anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                    sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                    roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                            main_side_let = main_side_let, ww=6,hh=3,
                            beh_states_to_shade = beh_states_to_shade, 
                             extdat_pri=d['emgdat_pri'])

    plt.tight_layout()
    plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,label_str,fsh)}_{rng}.pdf'))
    plt.close()

In [ ]:
len(collect_SHAP_outs)

# interpretML

In [ ]:
#for prefix_cur in prefixes:
#for prefix_cur in prefixes[:3]:
#for prefix_cur in prefixes:
for prefix in ['onlyMotorSrc']:
    prefixes_to_use = [prefix_cur]

    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
                                      ['merge_movements'],['trem_vs_hold&move'])
    print('      ',prefix_cur)
    for og in outputs_grouped.items():
        rn = og[0]
        (prefix,grp,int_type), mult_clf_output = og[1]
        assert prefix == prefix_cur
    
        filename_fullsize = mult_clf_output['filename_full']
        from pathlib import Path
        pfsz = Path(filename_fullsize)
        filename_fullsize = pjoin(pfsz.parents[0], pfsz.name[2:])
        #    finfo = os.stat( filename_fullsize )
        #    print(finfo.st_size / (1024**2))
        f = np.load(filename_fullsize,allow_pickle=True)

        results_cur =  f['results_cur'][()]

        #results_cur.keys()

        EBM = results_cur['featsel_per_method']['interpret_EBM']        
        #scores

        clf_dict = EBM['info_per_cp'][('trem_L', 'hold_L&move_L')]
        scores = clf_dict['scores']; 
        #print(len(results_cur['feature_names_filtered']), len(EBM['feature_indices_used']) )
        #print('len(scores) = ',len(scores) )
        explainer = clf_dict['explainer']
        
        print((rn,grp,int_type), utsne.sprintfPerfs(clf_dict['perf'] ) )

        del f
        del results_cur
        gc.collect()

In [ ]:
#       LFPrel_noself
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.70%,97.30%,73.97%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 54.75%,93.76%,65.06%
#        LFPrel_noself_onlyBpcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.09%,96.59%,64.78%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 49.71%,93.22%,60.30%
#        LFPrel_noself_onlyCon
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 42.19%,94.97%,53.35%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 30.51%,92.18%,41.29%
#        LFPrel_noself_onlyRbcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 48.94%,97.15%,62.01%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.56%,92.61%,46.82%
#        allb_beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 64.64%,95.34%,72.04%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.94%,90.69%,63.34%
#        cross_freqmod_beta,gamma:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 18.93%,97.11%,29.55%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 26.17%,94.16%,37.62%
#        cross_freqmod_tremor,beta:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 40.86%,95.38%,52.56%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.36%,92.31%,46.41%
#        cross_freqmod_tremor,gamma:beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 39.69%,96.07%,52.15%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 27.67%,94.66%,39.64%
#        modLFP
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 35.31%,95.57%,47.26%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 4.50%,98.84%,8.41%
#        modSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 72.72%,97.23%,80.13%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 62.01%,93.29%,70.18%
#        modSrc_self
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.11%,95.55%,71.22%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.49%,90.39%,62.74%
#        onlyCBSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.38%,95.51%,63.69%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 22.85%,94.62%,33.94%
#        onlyH
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 62.04%,95.14%,69.86%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 51.43%,91.03%,60.01%
#        onlyMotorSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 74.52%,97.34%,81.46%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 63.63%,94.94%,72.74%
#        onlyRestSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 66.35%,96.35%,74.51%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 60.19%,92.75%,68.41%

In [ ]:
(rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
     scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]

print(rn_,prefix,grp,int_type)
mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_off']['onlyH'][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_on']['onlyMotorSrc'][grp][int_type]

In [ ]:
print(range(0,6))

In [ ]:
[4  for d in [4,5] if d in [4,5]]

In [ ]:
help(plotFeatsAndRelDat)

In [ ]:
del plotFeatsAndRelDat

In [ ]:
import plots
import importlib as il; il.reload(plots)

In [ ]:
runMLvars['wbd_pri'][0]

In [ ]:
extdat_out_pri[0].shape

In [ ]:
from globvars import gp
beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

# TODO: right now these are not correct scores (perhaps) because of the way they are averaged
#%debug
rng = slice(0,8)
#rng = slice(0,2)
rng_feats = slice(0,2)
#rng_feats = slice(0,1)
from plots import plotFeatsAndRelDat
axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                         featnames_VIFsel[featis[rng_feats]], dat_out_pri,
                   subfeature_order_out_pri,
                runMLvars['X_pri'],[featnames_best_LFP]*len(rawnames_out),
                times_out_pri,
                runMLvars['Xtimes_pri'],
                subfeature_order_newsrcgrp_pri, runMLvars['wbd_pri'],
                dat_hires_pri=dat_lfp_hires_out_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_out_pri,
                times_hires_pri=times_hires_out_pri,
                anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                        main_side_let = main_side_let, ww=6,hh=3,
                        beh_states_to_shade = beh_states_to_shade, 
                         extdat_pri=emgdat_pri)

plt.tight_layout()
plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,fsh)}_{rng}.pdf'))
plt.close()

In [ ]:
best_feat_name

In [ ]:
print( utsne.sprintfPerfs(clf_dict['perf'] ) )
print( clf_dict['confmat_normalized'] )

In [ ]:
np.argmax(scores)

In [ ]:
assert len( explainer.feature_names ) == len(scores)
best_feat_name = explainer.feature_names [ np.argmax(scores)]
print(best_feat_name)
sortinds = np.argsort(scores)
last_ind = np.where([ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in  sortinds])[0][-1]
last_ind = sortinds[last_ind]
best_feat_name_no_int = explainer.feature_names[last_ind]
print(best_feat_name_no_int)

In [ ]:
sortinds[-1]

In [ ]:
 len(scores), last_ind

In [ ]:
explainer.feature_names[last_ind], m[last_ind], best_feat_name_no_int.find(' x ')

In [ ]:
np.where(m)[0][-1]

In [ ]:
m = [ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in np.argsort(scores) ]
print(m[344])

In [ ]:
print( explainer.__dict__.keys() )
# print( explainer.explanation_type, explainer.name, 
#      explainer.selector, explainer.feature_types)

In [ ]:
mult_clf_output['featsel_per_method']['interpret_EBM']['all'].keys()

from interpret import preserve
explainer = clf_dict['explainer']
preserve(explainer, best_feat_name_no_int)

In [ ]:
preserve(explainer, best_feat_name)

In [ ]:
explainer.visualize()

In [ ]:
# from interpret import show
# show(explainer)  # does not work if open saved

In [ ]:
# Trying to get a better distr fit for Hjorth (no success)

In [ ]:
len(featnames_nice), len(featnames_VIFsel)

In [ ]:
np.array(featnames_nice)[featis[-2:]]

In [ ]:
featnames[featis[-2:]]

In [ ]:
roi_labels['all_raw'][30]

In [ ]:
len(featnames_nice), len(featnames)

In [ ]:
featnames_nice[-5:], featnames[-5:]

In [ ]:
pjoin(subdir,'good_feats_dyn.pdf')

In [ ]:
# from plots import plotFeatsWithEverything
# #%debug
# plotFeatsWithEverything(dat_pri, rawtimes_pri, X_pri, 
#                             Xtimes_pri, dat_lfp_hires_pri, times_hires_pri,
#                             rawnames,
#                             subfeature_order_pri, subfeature_order_newsrcgrp_pri,
#                             subfeature_order_lfp_hires_pri,
#                             anndict_per_intcat_per_rawn,
#                             featnames, wbd_pri,
#                             sfreq, raw_perband_flt_pri, raw_perband_bp_pri,
#                             scale_data_combine_type,
#                             stats_multiband_flt, stats_multiband_bp,
#                             test_plots_descr, special_chns,
#                            fband_names_inc_HFO )

In [ ]:
np.min(Xconcat), np.max(Xconcat_to_fit)

In [ ]:
dd0 = Xconcat[:,featis[0]]
#dd = dd[ dd< np.quantile(dd,0.95)]
dd = np.exp(dd0) 
#dd = 1/dd
dd = np.log( dd - np.min(dd) *.99 )
#dd = dd[ dd< np.quantile(dd,0.99)]
plt.hist(dd,bins=250,alpha=0.5,label='new');

plt.hist(dd0,bins=250,alpha=0.5,label='old');
plt.legend(loc='upper right')

In [ ]:
plt.hist(dd0,bins=250)

In [ ]:
np.min(dd)

In [ ]:
h

In [ ]:
help(dist.pdf)

In [ ]:
f

In [ ]:
import scipy.stats
qm = np.quantile(dd,0.99)
y = dd [ dd < qm]
#y = np.exp(y)
size = len(y)
x = np.arange(np.min(y),np.max(y),0.5)
#y = 0.01 -np.min(dd) + dd
#x = np.arange(0.01,55,0.5)
#y = scipy.int_(np.round_(scipy.stats.vonmises.rvs(5,size=size)*47))
h = plt.hist(y, bins=x)
binh,binloc,barcont = h

# beta, rayleigh look ok
#ten best
# dist_names = ['cauchy',
#  'dweibull',
#  'foldcauchy',
#  'dgamma',
#  't',
#  'tukeylambda',
#  'fisk',
#  'weibull_min', 'erlang']

#'loglaplace',
 
dist_names = ['alpha', 'cauchy',  'dgamma', 'dweibull', 'erlang', 'exponweib',
              'genlogistic',  'foldcauchy',  'genextreme']
#  'fisk',  'fatiguelife',  'gumbel_r',  'gengamma','invweibull',
#  'invgamma',  'mielke',  'powerlognorm',  'nct',
#  'johnsonsu',  'lognorm',  'invgauss',  'loglaplace',
#  'recipinvgauss',  'pearson3',  'laplace', 't',  'tukeylambda',
#  'hypsecant',  'weibull_min',  'logistic',  'kstwobign',
#               'nakagami',  'rayleigh',  'rice',  'maxwell']
dist_names = ['beta']
dist_names = ['levy_stable']

do_plot = 1
d_per_name = {}
pdf_fit_per_name = {}

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    if arg:
        pdf_fitted = dist.pdf(binloc, *arg, loc=loc, scale=scale) * size
    else:
        pdf_fitted = dist.pdf(binloc, loc=loc, scale=scale) * size
    if do_plot and np.max(pdf_fitted) < np.max(binh) * 3:
        plt.plot(binloc, pdf_fitted, label=dist_name)
    plt.xlim(-6,20)
    pdf_fit_per_name[dist_name] = pdf_fitted
    
    discrep = np.sum(np.abs( binh-pdf_fitted[:-1] ) ) / len(binh)
    d_per_name[dist_name] = discrep
plt.legend(loc='upper right')
plt.show()

In [ ]:
arg

In [ ]:
help(sorted)

In [ ]:
stl = sorted( list( d_per_name.items() ), key = lambda x: x[1] )
stl_ext = [ (dn,v,pdf_fit_per_name[dn][0]) for  (dn, v) in stl]
display(stl_ext)

In [ ]:
list( set( ['exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 
               'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 
               'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 
               'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy'] ) - \
set( ['frechet_r',
 'genhalflogistic',
 'foldnorm',
 'gilbrat',
 'gompertz',
 'genpareto',
 'genexpon',
 'halfcauchy',
 'exponpow',
 'gamma',
 'frechet_l',
 'f',
 'gausshyper',
 'gumbel_l'] ) )

In [ ]:
list(list(zip(*stl))[0])[:10]

In [ ]:
list(list(zip(*stl))[0])[-14:]

In [ ]:
list(list(zip(*stl))[0])[-10:]

In [ ]:
len(binh), len(pdf_fitted)

In [ ]:
plt.plot(binh)
plt.plot(pdf_fitted[:-1])

In [ ]:
help(plt.subplots_adjust)